In [ ]:
pip install unidecode

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
from matplotlib.pyplot import hist
from unidecode import unidecode
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import pylab as P
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math

In [ ]:
encoding='ISO-8859-1'

In [ ]:
path = "/content/drive/MyDrive/314Final/code model/tranfer.csv"
df = pd.read_csv(path, encoding='ISO-8859-1')

In [ ]:
df

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,country
0,Atalanta BC,Alessandro Marchini,20.0,Central Midfield,Seregno,free transfer,out,Summer,0.0,Serie A,2016,2016/2017,Italy
1,Atalanta BC,Dávid Forgács,20.0,Left-Back,Ancona,free transfer,out,Summer,0.0,Serie A,2016,2016/2017,Italy
2,Cagliari Calcio,Alessandro Masia,20.0,Attacking Midfield,Tuttocuoio,free transfer,out,Summer,0.0,Serie A,2016,2016/2017,Italy
3,Chievo Verona,Oliver Posarelli,20.0,Centre-Back,MSU Billings,free transfer,out,Summer,0.0,Serie A,2016,2016/2017,Italy
4,Chievo Verona,Luis Maldonado,20.0,Defensive Midfield,Arzignano,free transfer,out,Summer,0.0,Serie A,2016,2016/2017,Italy
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11497,US Cremonese,Luka Lochoshvili,24.0,Centre-Back,Wolfsberger AC,1.60m,in,Summer,1.6,Serie A,2022,2022/2023,Italy
11498,US Cremonese,Jack Hendry,27.0,Centre-Back,Club Brugge,Loan fee:600Th.,in,Summer,0.6,Serie A,2022,2022/2023,Italy
11499,US Cremonese,Johan Vásquez,23.0,Centre-Back,Genoa,Loan fee:500Th.,in,Summer,0.5,Serie A,2022,2022/2023,Italy
11500,AC Monza,Andrea Carboni,21.0,Centre-Back,Cagliari Calcio,4.00m,in,Summer,4.0,Serie A,2022,2022/2023,Italy


# clean data

จัดการกับข้อมูล transfer ก่อน

ทำการลบ rows ของ column fee_cleaned ที่มีค่าเท่ากับ 0 และ column fee ที่มีค่าเป็น loan ออก

In [ ]:
for f in df['fee_cleaned']:
  if (f == 0.00 ) :
    df.drop(index=df[df['fee_cleaned'] == f].index, inplace=True)


for f in df['fee']:
  if ('loan' in f.lower()) :
    df.drop(index=df[df['fee'] == f].index, inplace=True)

สร้าง column short_name จาก player_name
เช่น Alexandre Pato => A. Pato

In [ ]:
df['short_name'] = df['player_name'].apply(unidecode)
for i, row in df.iterrows():
  temp = df.at[i,'short_name'].split()
  if(len(temp) > 1):
    df.at[i,'short_name'] = df.at[i,'short_name'].replace(temp[0], temp[0][0] + ".")

In [ ]:
df

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,country,short_name
4055,Villarreal CF,Alexandre Pato,27.0,Centre-Forward,TJ Quanjian,18.00m,out,Winter,18.0,Primera Division,2016,2016/2017,Spain,A. Pato
4056,Real Betis Balompié,Alin Tosca,24.0,Centre-Back,Steaua Bucharest,1.00m,in,Winter,1.0,Primera Division,2016,2016/2017,Spain,A. Tosca
4057,Sevilla FC,Walter Montoya,23.0,Right Winger,Rosario Central,5.50m,in,Winter,5.5,Primera Division,2016,2016/2017,Spain,W. Montoya
4058,Sevilla FC,Clément Lenglet,21.0,Centre-Back,AS Nancy,5.40m,in,Winter,5.4,Primera Division,2016,2016/2017,Spain,C. Lenglet
4059,Sevilla FC,Timothée Kolodziejczak,25.0,Centre-Back,Bor. M'gladbach,7.50m,out,Winter,7.5,Primera Division,2016,2016/2017,Spain,T. Kolodziejczak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11494,US Salernitana 1919,Dylan Bronn,27.0,Centre-Back,FC Metz,1.00m,in,Summer,1.0,Serie A,2022,2022/2023,Italy,D. Bronn
11496,US Cremonese,Emanuel Aiwu,21.0,Centre-Back,Rapid Vienna,3.50m,in,Summer,3.5,Serie A,2022,2022/2023,Italy,E. Aiwu
11497,US Cremonese,Luka Lochoshvili,24.0,Centre-Back,Wolfsberger AC,1.60m,in,Summer,1.6,Serie A,2022,2022/2023,Italy,L. Lochoshvili
11500,AC Monza,Andrea Carboni,21.0,Centre-Back,Cagliari Calcio,4.00m,in,Summer,4.0,Serie A,2022,2022/2023,Italy,A. Carboni


แยกข้อมูลออกมาแต่ละปี ตั้งแต่ปี 2016-2022

In [ ]:
df_2016 = df[df.year == 2016]
df_2017 = df[df.year == 2017]
df_2018 = df[df.year == 2018]
df_2019 = df[df.year == 2019]
df_2020 = df[df.year == 2020]
df_2021 = df[df.year == 2021]
df_2022 = df[df.year == 2022]

In [ ]:
df_2017

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season,country,short_name
4073,FC Barcelona,Philippe Coutinho,25.0,Left Winger,Liverpool,135.00m,in,Winter,135.000,Primera Division,2017,2017/2018,Spain,P. Coutinho
4074,FC Barcelona,Yerry Mina,23.0,Centre-Back,Palmeiras,12.40m,in,Winter,12.400,Primera Division,2017,2017/2018,Spain,Y. Mina
4075,FC Barcelona,Javier Mascherano,33.0,Defensive Midfield,HB CFFC,5.50m,out,Winter,5.500,Primera Division,2017,2017/2018,Spain,J. Mascherano
4077,Atlético de Madrid,Diego Costa,29.0,Centre-Forward,Chelsea,60.00m,in,Winter,60.000,Primera Division,2017,2017/2018,Spain,D. Costa
4078,Atlético de Madrid,Yannick Carrasco,24.0,Left Winger,DL Yifang,30.00m,out,Winter,30.000,Primera Division,2017,2017/2018,Spain,Y. Carrasco
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,Torino FC,Marco Chiosa,23.0,Centre-Back,Novara,500Th.,out,Summer,0.500,Serie A,2017,2017/2018,Italy,M. Chiosa
11246,Udinese Calcio,Bram Nuytinck,27.0,Centre-Back,RSC Anderlecht,3.00m,in,Summer,3.000,Serie A,2017,2017/2018,Italy,B. Nuytinck
11247,Udinese Calcio,Francisco Sierralta,20.0,Centre-Back,Granada CF,666Th.,in,Summer,0.666,Serie A,2017,2017/2018,Italy,F. Sierralta
11248,Udinese Calcio,Molla Wagué,26.0,Centre-Back,Granada CF,525Th.,in,Summer,0.525,Serie A,2017,2017/2018,Italy,M. Wague


import ข้อมูลนักเตะแต่ละคนตามปี

In [ ]:
file_path1 = "/content/drive/MyDrive/314Final/code model/all16.csv"
file_path2 = "/content/drive/MyDrive/314Final/code model/all17.csv"
file_path3 = "/content/drive/MyDrive/314Final/code model/all18.csv"
file_path4 = "/content/drive/MyDrive/314Final/code model/all19.csv"
file_path5 = "/content/drive/MyDrive/314Final/code model/all20.csv"
file_path6 = "/content/drive/MyDrive/314Final/code model/all21.csv"
file_path7 = "/content/drive/MyDrive/314Final/code model/all22.csv"


df16 = pd.read_csv(file_path1, encoding=encoding)
df17 = pd.read_csv(file_path2, encoding=encoding)
df18 = pd.read_csv(file_path3, encoding=encoding)
df19 = pd.read_csv(file_path4, encoding=encoding)
df20 = pd.read_csv(file_path5, encoding=encoding)
df21 = pd.read_csv(file_path6, encoding=encoding)
df22 = pd.read_csv(file_path7, encoding=encoding)

In [ ]:
#Array สำหรับเปรียบเทียบตัวเลขหน้าชื่อนักเตะ
temp_num = []
for i in range(50):
  if i < 10:
    temp_num.append('0'+str(i))
  else:
    temp_num.append(str(i))

temp_num

['00',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49']

In [ ]:
#ถอด accent latin ในชื่อนักเตะออก á -> a
df16['cleaned_name'] = df16['Name'].apply(unidecode)

#loop แก้ชื่อนักเตะ
# 1. ถอดตัวเลขออก "07 A. Censori"	-> "A. Censori"
# 2. ถอดช่องว่างหน้าชื่อนักเตะบางคน " L. Suárez" -> "L. Suárez"
for i, row in df16.iterrows():

  temp = df16.at[i,'cleaned_name'].split()
  num_in_name = 0
  if(temp[0] in temp_num) :
    num_in_name = 1
    df16.at[i,'cleaned_name'] = ' '.join(temp[1:])
  if(len(temp) - num_in_name >1):
    df16.at[i,'cleaned_name'] = df16.at[i,'cleaned_name'].replace(temp[0+ num_in_name], temp[0+ num_in_name][0] + ".")
  else:
    df16.at[i,'cleaned_name'] = df16.at[i,'cleaned_name'].replace(temp[0+ num_in_name], temp[0+ num_in_name])

In [ ]:
df16

,Id,Name,Team,Nationality,Position,Age,Overall,Potential,Value,Wage,...,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,cleaned_name
0,186238,J. Kean,Norwich City,England,GK,24,62,68,325000,4000,...,NaN,18,13,13,63,55,46,64,66,J. Kean
1,139691,M. Pugliesi,Empoli,Italy,GK,38,62,62,50000,2000,...,NaN,15,16,19,66,48,70,66,67,M. Pugliesi
2,199812,R. Allsop,AFC Bournemouth,England,GK,23,63,70,400000,4000,...,NaN,20,11,11,64,58,65,60,67,R. Allsop
3,201439,M. Mangiapelo,Frosinone,Italy,GK,29,63,63,275000,4000,...,NaN,12,19,18,60,60,55,65,70,M. Mangiapelo
4,206075,S. Johnstone,Manchester United,England,GK,22,63,73,475000,6000,...,NaN,13,17,20,69,56,53,60,72,S. Johnstone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2779,2,G. Pasquale,Udinese Calcio,Italy,LM,33,69,69,625000,7000,...,NaN,70,74,74,14,5,15,10,12,G. Pasquale
2780,11808,G. Bellini,Atalanta,Italy,"RB, LB",35,69,69,150000,7000,...,NaN,74,75,76,15,14,15,11,13,G. Bellini
2781,193974,M. Ciofani,Frosinone,Italy,"RB, CB, LB",27,68,69,775000,10000,...,NaN,69,72,72,5,9,5,9,11,M. Ciofani
2782,120018,D. Hammond,Leicester City,England,"CM, CDM",32,68,68,650000,8000,...,NaN,53,66,65,8,11,15,15,10,D. Hammond


In [ ]:
#ถอด accent latin ในชื่อนักเตะออก á -> a
df17['cleaned_name'] = df17['Name'].apply(unidecode)

#loop แก้ชื่อนักเตะ
# 1. ถอดตัวเลขออก "07 A. Censori"	-> "A. Censori"
# 2. ถอดช่องว่างหน้าชื่อนักเตะบางคน " L. Suárez" -> "L. Suárez"
for i, row in df17.iterrows():

  temp = df17.at[i,'cleaned_name'].split()
  num_in_name = 0
  if(temp[0] in temp_num) :
    num_in_name = 1
    df17.at[i,'cleaned_name'] = ' '.join(temp[1:])
  if(len(temp) - num_in_name >1):
    df17.at[i,'cleaned_name'] = df17.at[i,'cleaned_name'].replace(temp[0+ num_in_name], temp[0+ num_in_name][0] + ".")
  else:
    df17.at[i,'cleaned_name'] = df17.at[i,'cleaned_name'].replace(temp[0+ num_in_name], temp[0+ num_in_name])

In [ ]:
#ถอด accent latin ในชื่อนักเตะออก á -> a
df18['cleaned_name'] = df18['Name'].apply(unidecode)

#loop แก้ชื่อนักเตะ
# 1. ถอดตัวเลขออก "07 A. Censori"	-> "A. Censori"
# 2. ถอดช่องว่างหน้าชื่อนักเตะบางคน " L. Suárez" -> "L. Suárez"
for i, row in df18.iterrows():

  temp = df18.at[i,'cleaned_name'].split()
  num_in_name = 0
  if(temp[0] in temp_num) :
    num_in_name = 1
    df18.at[i,'cleaned_name'] = ' '.join(temp[1:])
  if(len(temp) - num_in_name >1):
    df18.at[i,'cleaned_name'] = df18.at[i,'cleaned_name'].replace(temp[0+ num_in_name], temp[0+ num_in_name][0] + ".")
  else:
    df18.at[i,'cleaned_name'] = df18.at[i,'cleaned_name'].replace(temp[0+ num_in_name], temp[0+ num_in_name])

In [ ]:
#ถอด accent latin ในชื่อนักเตะออก á -> a
df19['cleaned_name'] = df19['Name'].apply(unidecode)

#loop แก้ชื่อนักเตะ
# 1. ถอดตัวเลขออก "07 A. Censori"	-> "A. Censori"
# 2. ถอดช่องว่างหน้าชื่อนักเตะบางคน " L. Suárez" -> "L. Suárez"
for i, row in df16.iterrows():

  temp = df19.at[i,'cleaned_name'].split()
  num_in_name = 0
  if(temp[0] in temp_num) :
    num_in_name = 1
    df19.at[i,'cleaned_name'] = ' '.join(temp[1:])
  if(len(temp) - num_in_name >1):
    df19.at[i,'cleaned_name'] = df19.at[i,'cleaned_name'].replace(temp[0+ num_in_name], temp[0+ num_in_name][0] + ".")
  else:
    df19.at[i,'cleaned_name'] = df19.at[i,'cleaned_name'].replace(temp[0+ num_in_name], temp[0+ num_in_name])

In [ ]:
#ถอด accent latin ในชื่อนักเตะออก á -> a
df20['cleaned_name'] = df20['Name'].apply(unidecode)

#loop แก้ชื่อนักเตะ
# 1. ถอดตัวเลขออก "07 A. Censori"	-> "A. Censori"
# 2. ถอดช่องว่างหน้าชื่อนักเตะบางคน " L. Suárez" -> "L. Suárez"
for i, row in df20.iterrows():

  temp = df20.at[i,'cleaned_name'].split()
  num_in_name = 0
  if(temp[0] in temp_num) :
    num_in_name = 1
    df20.at[i,'cleaned_name'] = ' '.join(temp[1:])
  if(len(temp) - num_in_name >1):
    df20.at[i,'cleaned_name'] = df20.at[i,'cleaned_name'].replace(temp[0+ num_in_name], temp[0+ num_in_name][0] + ".")
  else:
    df20.at[i,'cleaned_name'] = df20.at[i,'cleaned_name'].replace(temp[0+ num_in_name], temp[0+ num_in_name])

In [ ]:
#ถอด accent latin ในชื่อนักเตะออก á -> a
df21['cleaned_name'] = df21['Name'].apply(unidecode)

#loop แก้ชื่อนักเตะ
# 1. ถอดตัวเลขออก "07 A. Censori"	-> "A. Censori"
# 2. ถอดช่องว่างหน้าชื่อนักเตะบางคน " L. Suárez" -> "L. Suárez"
for i, row in df21.iterrows():

  temp = df21.at[i,'cleaned_name'].split()
  num_in_name = 0
  if(temp[0] in temp_num) :
    num_in_name = 1
    df21.at[i,'cleaned_name'] = ' '.join(temp[1:])
  if(len(temp) - num_in_name >1):
    df21.at[i,'cleaned_name'] = df21.at[i,'cleaned_name'].replace(temp[0+ num_in_name], temp[0+ num_in_name][0] + ".")
  else:
    df21.at[i,'cleaned_name'] = df21.at[i,'cleaned_name'].replace(temp[0+ num_in_name], temp[0+ num_in_name])

In [ ]:
#ถอด accent latin ในชื่อนักเตะออก á -> a
df22['cleaned_name'] = df22['Name'].apply(unidecode)

#loop แก้ชื่อนักเตะ
# 1. ถอดตัวเลขออก "07 A. Censori"	-> "A. Censori"
# 2. ถอดช่องว่างหน้าชื่อนักเตะบางคน " L. Suárez" -> "L. Suárez"
for i, row in df22.iterrows():

  temp = df22.at[i,'cleaned_name'].split()
  num_in_name = 0
  if(temp[0] in temp_num) :
    num_in_name = 1
    df22.at[i,'cleaned_name'] = ' '.join(temp[1:])
  if(len(temp) - num_in_name >1):
    df22.at[i,'cleaned_name'] = df22.at[i,'cleaned_name'].replace(temp[0+ num_in_name], temp[0+ num_in_name][0] + ".")
  else:
    df22.at[i,'cleaned_name'] = df22.at[i,'cleaned_name'].replace(temp[0+ num_in_name], temp[0+ num_in_name])

รวมชุดข้อมูลทั้ง2เข้าด้วยกัน

In [ ]:
left = df_2016
right = df16
right.rename({ 'cleaned_name': 'short_name', 'age':'age'}, axis=1, inplace=True)
result_2016 = pd.merge(left, right, how="inner", on=['short_name'])

In [ ]:
df16

,Id,Name,Team,Nationality,Position,Age,Overall,Potential,Value,Wage,...,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,short_name
0,186238,J. Kean,Norwich City,England,GK,24,62,68,325000,4000,...,NaN,18,13,13,63,55,46,64,66,J. Kean
1,139691,M. Pugliesi,Empoli,Italy,GK,38,62,62,50000,2000,...,NaN,15,16,19,66,48,70,66,67,M. Pugliesi
2,199812,R. Allsop,AFC Bournemouth,England,GK,23,63,70,400000,4000,...,NaN,20,11,11,64,58,65,60,67,R. Allsop
3,201439,M. Mangiapelo,Frosinone,Italy,GK,29,63,63,275000,4000,...,NaN,12,19,18,60,60,55,65,70,M. Mangiapelo
4,206075,S. Johnstone,Manchester United,England,GK,22,63,73,475000,6000,...,NaN,13,17,20,69,56,53,60,72,S. Johnstone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2779,2,G. Pasquale,Udinese Calcio,Italy,LM,33,69,69,625000,7000,...,NaN,70,74,74,14,5,15,10,12,G. Pasquale
2780,11808,G. Bellini,Atalanta,Italy,"RB, LB",35,69,69,150000,7000,...,NaN,74,75,76,15,14,15,11,13,G. Bellini
2781,193974,M. Ciofani,Frosinone,Italy,"RB, CB, LB",27,68,69,775000,10000,...,NaN,69,72,72,5,9,5,9,11,M. Ciofani
2782,120018,D. Hammond,Leicester City,England,"CM, CDM",32,68,68,650000,8000,...,NaN,53,66,65,8,11,15,15,10,D. Hammond


In [ ]:
result_2016

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,Sevilla FC,Timothée Kolodziejczak,25.0,Centre-Back,Bor. M'gladbach,7.50m,out,Winter,7.5,Primera Division,...,49,NaN,71,77,76,14,6,10,16,7
1,Borussia Mönchengladbach,Timothée Kolodziejczak,25.0,Centre-Back,Sevilla FC,7.50m,in,Winter,7.5,1 Bundesliga,...,49,NaN,71,77,76,14,6,10,16,7
2,Sevilla FC,Hiroshi Kiyotake,27.0,Left Midfield,Cerezo Osaka,5.10m,out,Winter,5.1,Primera Division,...,66,NaN,23,24,27,15,10,12,7,10
3,Sevilla FC,Hiroshi Kiyotake,26.0,Left Midfield,Hannover 96,6.50m,in,Summer,6.5,Primera Division,...,66,NaN,23,24,27,15,10,12,7,10
4,Real Sociedad,Gerónimo Rulli,24.0,Goalkeeper,Man City,7.00m,in,Winter,7.0,Primera Division,...,24,NaN,16,13,17,82,77,70,78,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,SS Lazio,Wallace,21.0,Centre-Back,SC Braga,8.00m,in,Summer,8.0,Serie A,...,60,NaN,65,69,72,13,14,8,11,13
535,Delfino Pescara 1936,Andrea Coda,31.0,Centre-Back,Sampdoria,200Th.,in,Summer,0.2,Serie A,...,38,NaN,74,74,72,8,9,14,15,6
536,UC Sampdoria,Andrea Coda,31.0,Centre-Back,Pescara,200Th.,out,Summer,0.2,Serie A,...,38,NaN,74,74,72,8,9,14,15,6
537,AS Roma,Antonio Rüdiger,23.0,Centre-Back,VfB Stuttgart,9.00m,in,Summer,9.0,Serie A,...,40,NaN,77,79,74,14,15,10,12,8


In [ ]:
column_names = result_2016.columns
print(column_names)

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Nationality', 'Position', 'Age', 'Overall', 'Potential',
       'Value', 'Wage', 'preferred_foot', 'Work Rate', 'Body Type', 'Crossing',
       'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes'],
      dtype='object')


In [ ]:
result_2016 = result_2016.drop(columns=['Age'])


In [ ]:
column_names = result_2016.columns
print(column_names)

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Nationality', 'Position', 'Overall', 'Potential', 'Value',
       'Wage', 'preferred_foot', 'Work Rate', 'Body Type', 'Crossing',
       'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes'],
      dtype='object')


In [ ]:
#result_temp_2016=result_2016[result_2016.Overall.isnull()]

In [ ]:
#print(result_temp_2016.columns)

In [ ]:
#result_temp_2016

In [ ]:
#result_temp_2016 = result_temp_2016[list(df)]
#result_temp_2016.info()

In [ ]:
#result_temp_2016['age'] =  result_temp_2016['age']-1

In [ ]:
#left = result_temp_2016
#right = df16
#right.rename({'cleaned_name': 'short_name', 'Age':'age'}, axis=1, inplace=True)
#result_2016_new = pd.merge(left, right, how="inner", on=['short_name'])

In [ ]:
#result_2016_new

In [ ]:
result_2016.to_csv('result_2016.csv',encoding='utf-8-sig')

In [ ]:
left = df_2017
right = df17
right.rename({ 'cleaned_name': 'short_name', 'age':'age'}, axis=1, inplace=True)
result_2017 = pd.merge(left, right, how="inner", on=['short_name'])

In [ ]:
result_2017

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name_x,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,FC Barcelona,Javier Mascherano,33.0,Defensive Midfield,HB CFFC,5.50m,out,Winter,5.500,Primera Division,...,60,86,84,87,85,6,10,8,5,5
1,Atlético de Madrid,Diego Costa,29.0,Centre-Forward,Chelsea,60.00m,in,Winter,60.000,Primera Division,...,23,43,16,19,18,70,68,63,73,74
2,Atlético de Madrid,Diego Costa,29.0,Centre-Forward,Chelsea,60.00m,in,Winter,60.000,Primera Division,...,76,84,28,39,34,11,13,12,8,11
3,Atlético de Madrid,Diego Costa,29.0,Centre-Forward,Chelsea,60.00m,in,Winter,60.000,Primera Division,...,77,73,41,58,44,13,15,9,12,5
4,Chelsea FC,Diego Costa,29.0,Centre-Forward,Atlético Madrid,60.00m,out,Winter,60.000,Premier League,...,23,43,16,19,18,70,68,63,73,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,AC Milan,Leonardo Bonucci,30.0,Centre-Back,Juventus,42.00m,in,Summer,42.000,Serie A,...,70,82,84,88,88,2,2,3,2,4
580,US Sassuolo,Edoardo Goldaniga,23.0,Centre-Back,US Palermo,4.00m,in,Summer,4.000,Serie A,...,34,42,74,73,70,9,12,7,15,15
581,SPAL 2013,Felipe,32.0,Centre-Back,Udinese Calcio,600Th.,in,Summer,0.600,Serie A,...,35,67,80,80,76,11,6,8,9,5
582,Udinese Calcio,Felipe,32.0,Centre-Back,SPAL,600Th.,out,Summer,0.600,Serie A,...,35,67,80,80,76,11,6,8,9,5


In [ ]:
column_names = result_2017.columns
print(column_names)

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name_x', 'year', 'season', 'country', 'short_name', 'Id',
       'Name', 'Team', 'Positions', 'Overall', 'Potential', 'Value', 'Wage',
       'Age', 'league_name_y', 'Nationality', 'preferred_foot', 'Work Rate',
       'Body Type', 'Crossing', 'Finishing', 'Heading accuracy',
       'Short passing', 'Volleys', 'Dribbling', 'Curve', 'Fk accuracy',
       'Long passing', 'Ball control', 'Acceleration', 'Sprint speed',
       'Agility', 'Reactions', 'Balance', 'Shot power', 'Jumping', 'Stamina',
       'Strength', 'Long shots', 'Aggression', 'Interceptions', 'Positioning',
       'Vision', 'Penalties', 'Composure', 'Defensive awareness',
       'Standing tackle', 'Sliding tackle', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes'],
      dtype='object')


In [ ]:
result_2017 = result_2017.drop(columns=['Age'])

In [ ]:
result_2017 = result_2017.drop(columns=['league_name_y'])

In [ ]:
result_2017.rename(columns={'league_name_x': 'league_name'}, inplace=True)
result_2017.rename(columns={'Positions': 'Position'}, inplace=True)

In [ ]:
column_names = result_2017.columns
print(column_names)

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Position', 'Overall', 'Potential', 'Value', 'Wage',
       'Nationality', 'preferred_foot', 'Work Rate', 'Body Type', 'Crossing',
       'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes'],
      dtype='object')


In [ ]:
result_2017.to_csv('result_2017.csv',encoding='utf-8-sig')

In [ ]:
left = df_2018
right = df18
right.rename({ 'cleaned_name': 'short_name', 'age':'age'}, axis=1, inplace=True)
result_2018 = pd.merge(left, right, how="inner", on=['short_name'])

In [ ]:
result_2018

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name_x,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,Athletic Bilbao,Ibai Gómez,29.0,Right Winger,Alavés,4.00m,in,Winter,4.00,Primera Division,...,65,67,45,58,55,14,8,13,7,11
1,Deportivo Alavés,Ibai Gómez,29.0,Right Winger,Athletic,4.00m,out,Winter,4.00,Primera Division,...,65,67,45,58,55,14,8,13,7,11
2,Athletic Bilbao,Kenan Kodro,25.0,Centre-Forward,FC Copenhagen,900Th.,in,Winter,0.90,Primera Division,...,60,55,15,41,37,7,14,15,6,12
3,1.FSV Mainz 05,Kenan Kodro,24.0,Centre-Forward,FC Copenhagen,1.60m,out,Summer,1.60,1 Bundesliga,...,60,55,15,41,37,7,14,15,6,12
4,Deportivo Alavés,Rubén Sobrino,26.0,Centre-Forward,Valencia,5.00m,out,Winter,5.00,Primera Division,...,66,61,19,31,17,13,8,11,7,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,Juventus FC,Mattia Caldara,24.0,Centre-Back,AC Milan,36.87m,out,Summer,36.87,Serie A,...,30,73,78,79,76,14,9,6,9,5
503,AC Milan,Mattia Caldara,24.0,Centre-Back,Juventus,36.87m,in,Summer,36.87,Serie A,...,30,73,78,79,76,14,9,6,9,5
504,SS Lazio,Francesco Acerbi,30.0,Centre-Back,Sassuolo,12.00m,in,Summer,12.00,Serie A,...,72,83,86,86,85,11,9,9,13,11
505,US Sassuolo,Francesco Acerbi,30.0,Centre-Back,Lazio,12.00m,out,Summer,12.00,Serie A,...,72,83,86,86,85,11,9,9,13,11


In [ ]:
column_names = result_2018.columns
print(column_names)

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name_x', 'year', 'season', 'country', 'short_name', 'Id',
       'Name', 'Position', 'Overall', 'Potential', 'Value', 'Wage', 'Age',
       'Team', 'league_name_y', 'Nationality', 'preferred_foot', 'Work Rate',
       'Body type', 'Release cause', 'Crossing', 'Finishing',
       'Heading accuracy', 'Short passing', 'Volleys', 'Dribbling', 'Curve',
       'Fk accuracy', 'Long passing', 'Ball control', 'Acceleration',
       'Sprint speed', 'Agility', 'Reactions', 'Balance', 'Shot power',
       'Jumping', 'Stamina', 'Strength', 'Long shots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Defensive awareness', 'Standing tackle', 'Sliding tackle', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes'],
      dtype='object')


In [ ]:
result_2018 = result_2018.drop(columns=['Age'])
result_2018 = result_2018.drop(columns=['league_name_y'])
result_2018.rename(columns={'league_name_x': 'league_name'}, inplace=True)
result_2018.rename(columns={'Release cause': 'Release Clause'}, inplace=True)
result_2018.rename(columns={'Body type': 'Body Type'}, inplace=True)

In [ ]:
result_2018

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,Athletic Bilbao,Ibai Gómez,29.0,Right Winger,Alavés,4.00m,in,Winter,4.00,Primera Division,...,65,67,45,58,55,14,8,13,7,11
1,Deportivo Alavés,Ibai Gómez,29.0,Right Winger,Athletic,4.00m,out,Winter,4.00,Primera Division,...,65,67,45,58,55,14,8,13,7,11
2,Athletic Bilbao,Kenan Kodro,25.0,Centre-Forward,FC Copenhagen,900Th.,in,Winter,0.90,Primera Division,...,60,55,15,41,37,7,14,15,6,12
3,1.FSV Mainz 05,Kenan Kodro,24.0,Centre-Forward,FC Copenhagen,1.60m,out,Summer,1.60,1 Bundesliga,...,60,55,15,41,37,7,14,15,6,12
4,Deportivo Alavés,Rubén Sobrino,26.0,Centre-Forward,Valencia,5.00m,out,Winter,5.00,Primera Division,...,66,61,19,31,17,13,8,11,7,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,Juventus FC,Mattia Caldara,24.0,Centre-Back,AC Milan,36.87m,out,Summer,36.87,Serie A,...,30,73,78,79,76,14,9,6,9,5
503,AC Milan,Mattia Caldara,24.0,Centre-Back,Juventus,36.87m,in,Summer,36.87,Serie A,...,30,73,78,79,76,14,9,6,9,5
504,SS Lazio,Francesco Acerbi,30.0,Centre-Back,Sassuolo,12.00m,in,Summer,12.00,Serie A,...,72,83,86,86,85,11,9,9,13,11
505,US Sassuolo,Francesco Acerbi,30.0,Centre-Back,Lazio,12.00m,out,Summer,12.00,Serie A,...,72,83,86,86,85,11,9,9,13,11


In [ ]:
column_names = result_2018.columns
print(column_names)

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Position', 'Overall', 'Potential', 'Value', 'Wage', 'Team',
       'Nationality', 'preferred_foot', 'Work Rate', 'Body Type',
       'Release Clause', 'Crossing', 'Finishing', 'Heading accuracy',
       'Short passing', 'Volleys', 'Dribbling', 'Curve', 'Fk accuracy',
       'Long passing', 'Ball control', 'Acceleration', 'Sprint speed',
       'Agility', 'Reactions', 'Balance', 'Shot power', 'Jumping', 'Stamina',
       'Strength', 'Long shots', 'Aggression', 'Interceptions', 'Positioning',
       'Vision', 'Penalties', 'Composure', 'Defensive awareness',
       'Standing tackle', 'Sliding tackle', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes'],
      dtype='object')


In [ ]:
result_2018.to_csv('result_2018.csv',encoding='utf-8-sig')

In [ ]:
left = df_2019
right = df19
right.rename({ 'cleaned_name': 'short_name', 'age':'age'}, axis=1, inplace=True)
result_2019 = pd.merge(left, right, how="inner", on=['short_name'])

In [ ]:
result_2019

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name_x,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,Getafe CF,Erick Cabaco,24.0,Centre-Back,Levante,8.00m,in,Winter,8.0,Primera Division,...,32,66,77,75,75,11,7,13,14,6
1,Levante UD,Erick Cabaco,24.0,Centre-Back,Getafe,8.00m,out,Winter,8.0,Primera Division,...,32,66,77,75,75,11,7,13,14,6
2,Getafe CF,Chema Rodríguez,27.0,Centre-Back,Nottm Forest,1.50m,in,Winter,1.5,Primera Division,...,48,65,72,72,76,15,11,6,14,12
3,Levante UD,Chema Rodríguez,27.0,Centre-Back,Nottm Forest,500Th.,out,Summer,0.5,Primera Division,...,48,65,72,72,76,15,11,6,14,12
4,Getafe CF,Leandro Cabrera,28.0,Centre-Back,Espanyol,9.00m,out,Winter,9.0,Primera Division,...,49,56,55,75,78,10,16,10,15,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,US Lecce,Luca Rossettini,34.0,Centre-Back,Genoa,500Th.,in,Summer,0.5,Serie A,...,45,74,80,78,77,5,5,5,5,5
586,Genoa CFC,Nicolás Spolli,36.0,Centre-Back,Crotone,400Th.,out,Summer,0.4,Serie A,...,33,73,76,78,74,8,7,10,10,14
587,SSC Napoli,Konstantinos Manolas,28.0,Centre-Back,AS Roma,36.00m,in,Summer,36.0,Serie A,...,26,79,86,87,89,8,6,8,15,12
588,AS Roma,Konstantinos Manolas,28.0,Centre-Back,SSC Napoli,36.00m,out,Summer,36.0,Serie A,...,26,79,86,87,89,8,6,8,15,12


In [ ]:
column_names = result_2019.columns
print(column_names)

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name_x', 'year', 'season', 'country', 'short_name', 'Id',
       'Name', 'Team', 'Nationality', 'Position', 'Overall', 'Potential',
       'Value', 'Wage', 'Age', 'league_name_y', 'preferred_foot', 'Work Rate',
       'Body type', 'Release Cluase', 'Crossing', 'Finishing',
       'Heading accuracy', 'Short passing', 'Volleys', 'Dribbling', 'Curve',
       'Fk accuracy', 'Long passing', 'Ball control', 'Acceleration',
       'Sprint speed', 'Agility', 'Reactions', 'Balance', 'Shot power',
       'Jumping', 'Stamina', 'Strength', 'Long shots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Defensive awareness', 'Standing tackle', 'Sliding tackle', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes'],
      dtype='object')


In [ ]:
result_2019 = result_2019.drop(columns=['Age'])
result_2019 = result_2019.drop(columns=['league_name_y'])
result_2019.rename(columns={'league_name_x': 'league_name'}, inplace=True)
result_2019.rename(columns={'Body type': 'Body Type'}, inplace=True)
result_2019.rename(columns={'Release Cluase': 'Release Clause'}, inplace=True)

In [ ]:
column_names = result_2019.columns
print(column_names)

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Nationality', 'Position', 'Overall', 'Potential', 'Value',
       'Wage', 'preferred_foot', 'Work Rate', 'Body Type', 'Release Clause',
       'Crossing', 'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes'],
      dtype='object')


In [ ]:
result_2019

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,Getafe CF,Erick Cabaco,24.0,Centre-Back,Levante,8.00m,in,Winter,8.0,Primera Division,...,32,66,77,75,75,11,7,13,14,6
1,Levante UD,Erick Cabaco,24.0,Centre-Back,Getafe,8.00m,out,Winter,8.0,Primera Division,...,32,66,77,75,75,11,7,13,14,6
2,Getafe CF,Chema Rodríguez,27.0,Centre-Back,Nottm Forest,1.50m,in,Winter,1.5,Primera Division,...,48,65,72,72,76,15,11,6,14,12
3,Levante UD,Chema Rodríguez,27.0,Centre-Back,Nottm Forest,500Th.,out,Summer,0.5,Primera Division,...,48,65,72,72,76,15,11,6,14,12
4,Getafe CF,Leandro Cabrera,28.0,Centre-Back,Espanyol,9.00m,out,Winter,9.0,Primera Division,...,49,56,55,75,78,10,16,10,15,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,US Lecce,Luca Rossettini,34.0,Centre-Back,Genoa,500Th.,in,Summer,0.5,Serie A,...,45,74,80,78,77,5,5,5,5,5
586,Genoa CFC,Nicolás Spolli,36.0,Centre-Back,Crotone,400Th.,out,Summer,0.4,Serie A,...,33,73,76,78,74,8,7,10,10,14
587,SSC Napoli,Konstantinos Manolas,28.0,Centre-Back,AS Roma,36.00m,in,Summer,36.0,Serie A,...,26,79,86,87,89,8,6,8,15,12
588,AS Roma,Konstantinos Manolas,28.0,Centre-Back,SSC Napoli,36.00m,out,Summer,36.0,Serie A,...,26,79,86,87,89,8,6,8,15,12


In [ ]:
result_2019.to_csv('result_2019.csv',encoding='utf-8-sig')

In [ ]:
left = df_2020
right = df20
right.rename({ 'cleaned_name': 'short_name', 'age':'age'}, axis=1, inplace=True)
result_2020=pd.merge(left, right, how="inner", on=['short_name'])

In [ ]:
result_2020

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name_x,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,Atlético de Madrid,Geoffrey Kondogbia,27.0,Defensive Midfield,Valencia,15.00m,in,Winter,15.00,Primera Division,...,45,84,77,80,81,15,10,10,8,10
1,Valencia CF,Geoffrey Kondogbia,27.0,Defensive Midfield,Atlético Madrid,15.00m,out,Winter,15.00,Primera Division,...,45,84,77,80,81,15,10,10,8,10
2,Sevilla FC,Carlos Fernández,24.0,Centre-Forward,Real Sociedad,10.00m,out,Winter,10.00,Primera Division,...,70,66,21,21,22,12,13,8,9,12
3,Real Sociedad,Carlos Fernández,24.0,Centre-Forward,Sevilla FC,10.00m,in,Winter,10.00,Primera Division,...,70,66,21,21,22,12,13,8,9,12
4,Villarreal CF,Étienne Capoue,32.0,Defensive Midfield,Watford,2.00m,in,Winter,2.00,Primera Division,...,54,75,76,79,77,12,11,11,14,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,Torino FC,Ricardo Rodríguez,28.0,Centre-Back,AC Milan,1.95m,in,Summer,1.95,Serie A,...,83,80,80,78,78,14,13,14,14,11
364,SSC Napoli,Lorenzo Tonelli,30.0,Centre-Back,Sampdoria,2.00m,out,Summer,2.00,Serie A,...,35,74,79,81,79,13,11,13,11,6
365,UC Sampdoria,Lorenzo Tonelli,30.0,Centre-Back,SSC Napoli,2.00m,in,Summer,2.00,Serie A,...,35,74,79,81,79,13,11,13,11,6
366,US Sassuolo,Kaan Ayhan,25.0,Centre-Back,F. Düsseldorf,2.50m,in,Summer,2.50,Serie A,...,47,75,77,77,73,16,10,8,14,10


In [ ]:
column_names = result_2020.columns
print(column_names)

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name_x', 'year', 'season', 'country', 'short_name', 'Id',
       'Name', 'Team', 'Position', 'Overall', 'Potential', 'Value', 'Wage',
       'Age', 'league_name_y', 'Nationality', 'preferred_foot', 'Work rate',
       'Body type', 'Release Cluase', 'Crossing', 'Finishing',
       'Heading accuracy', 'Short passing', 'Volleys', 'Dribbling', 'Curve',
       'Fk accuracy', 'Long passing', 'Ball control', 'Acceleration',
       'Sprint speed', 'Agility', 'Reactions', 'Balance', 'Shot power',
       'Jumping', 'Stamina', 'Strength', 'Long shots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Defensive awareness', 'Standing tackle', 'Sliding tackle', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes'],
      dtype='object')


In [ ]:
result_2020 = result_2020.drop(columns=['Age'])
result_2020 = result_2020.drop(columns=['league_name_y'])
result_2020.rename(columns={'league_name_x': 'league_name'}, inplace=True)
result_2020.rename(columns={'Work rate': 'Work Rate'}, inplace=True)
result_2020.rename(columns={'Body type': 'Body Type'}, inplace=True)
result_2020.rename(columns={'Release Cluase': 'Release Clause'}, inplace=True)

In [ ]:
result_2020

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,Atlético de Madrid,Geoffrey Kondogbia,27.0,Defensive Midfield,Valencia,15.00m,in,Winter,15.00,Primera Division,...,45,84,77,80,81,15,10,10,8,10
1,Valencia CF,Geoffrey Kondogbia,27.0,Defensive Midfield,Atlético Madrid,15.00m,out,Winter,15.00,Primera Division,...,45,84,77,80,81,15,10,10,8,10
2,Sevilla FC,Carlos Fernández,24.0,Centre-Forward,Real Sociedad,10.00m,out,Winter,10.00,Primera Division,...,70,66,21,21,22,12,13,8,9,12
3,Real Sociedad,Carlos Fernández,24.0,Centre-Forward,Sevilla FC,10.00m,in,Winter,10.00,Primera Division,...,70,66,21,21,22,12,13,8,9,12
4,Villarreal CF,Étienne Capoue,32.0,Defensive Midfield,Watford,2.00m,in,Winter,2.00,Primera Division,...,54,75,76,79,77,12,11,11,14,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,Torino FC,Ricardo Rodríguez,28.0,Centre-Back,AC Milan,1.95m,in,Summer,1.95,Serie A,...,83,80,80,78,78,14,13,14,14,11
364,SSC Napoli,Lorenzo Tonelli,30.0,Centre-Back,Sampdoria,2.00m,out,Summer,2.00,Serie A,...,35,74,79,81,79,13,11,13,11,6
365,UC Sampdoria,Lorenzo Tonelli,30.0,Centre-Back,SSC Napoli,2.00m,in,Summer,2.00,Serie A,...,35,74,79,81,79,13,11,13,11,6
366,US Sassuolo,Kaan Ayhan,25.0,Centre-Back,F. Düsseldorf,2.50m,in,Summer,2.50,Serie A,...,47,75,77,77,73,16,10,8,14,10


In [ ]:
result_2020.to_csv('result_2020.csv',encoding='utf-8-sig')

In [ ]:
left = df_2021
right = df21
right.rename({ 'cleaned_name': 'short_name', 'age':'age'}, axis=1, inplace=True)
result_2021 = pd.merge(left, right, how="inner", on=['short_name'])

In [ ]:
result_2021

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name_x,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,Atlético de Madrid,Daniel Wass,32.0,Central Midfield,Valencia,2.70m,in,Winter,2.70,Primera Division,...,64,75,79,78,77,11,9,10,13,13
1,Valencia CF,Daniel Wass,32.0,Central Midfield,Atlético Madrid,2.70m,out,Winter,2.70,Primera Division,...,64,75,79,78,77,11,9,10,13,13
2,Atlético de Madrid,Kieran Trippier,31.0,Right-Back,Newcastle,30.00m,out,Winter,30.00,Primera Division,...,66,77,77,83,81,11,14,8,11,10
3,Newcastle United,Kieran Trippier,31.0,Right-Back,Atlético Madrid,30.00m,in,Winter,30.00,Premier League,...,66,77,77,83,81,11,14,8,11,10
4,FC Barcelona,Ferran Torres,21.0,Left Winger,Man City,55.00m,in,Winter,55.00,Primera Division,...,66,82,35,24,28,12,9,11,15,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,ACF Fiorentina,Matija Nastasi?,28.0,Centre-Back,FC Schalke 04,500Th.,in,Summer,0.50,Serie A,...,52,80,80,78,81,11,16,11,15,13
381,ACF Fiorentina,Julián Illanes,24.0,Centre-Back,Pescara,350Th.,out,Summer,0.35,Serie A,...,40,43,61,66,71,7,7,5,10,15
382,Spezia Calcio,Ardian Ismajli,24.0,Centre-Back,FC Empoli,2.00m,out,Summer,2.00,Serie A,...,31,62,71,72,70,9,14,10,12,13
383,FC Empoli,Ardian Ismajli,24.0,Centre-Back,Spezia Calcio,2.00m,in,Summer,2.00,Serie A,...,31,62,71,72,70,9,14,10,12,13


In [ ]:
column_names = result_2021.columns
print(column_names)

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name_x', 'year', 'season', 'country', 'short_name', 'Name',
       'Team', 'Position', 'Nationality', 'Overall', 'Potential', 'Value',
       'Wage', 'Age', 'league_name_y', 'preferred_foot', 'Work rate',
       'Bofy type', 'Release cluase', 'Crossing', 'Finishing',
       'Heading accuracy', 'Short passing', 'Volleys', 'Dribbling', 'Curve',
       'Fk accuracy', 'Long passing', 'Ball control', 'Acceleration',
       'Sprint speed', 'Agility', 'Reactions', 'Balance', 'Shot power',
       'Jumping', 'Stamina', 'Strength', 'Long shots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Defensive awareness', 'Standing tackle', 'Sliding tackle', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes'],
      dtype='object')


In [ ]:
result_2021 = result_2021.drop(columns=['league_name_y'])
result_2021.rename(columns={'league_name_x': 'league_name'}, inplace=True)
result_2021 = result_2021.drop(columns=['Age'])
result_2021.rename(columns={'Work rate': 'Work Rate'}, inplace=True)
result_2021.rename(columns={'Bofy type': 'Body Type'}, inplace=True)
result_2021.rename(columns={'Release cluase': 'Release Clause'}, inplace=True)

In [ ]:
result_2021

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,Atlético de Madrid,Daniel Wass,32.0,Central Midfield,Valencia,2.70m,in,Winter,2.70,Primera Division,...,64,75,79,78,77,11,9,10,13,13
1,Valencia CF,Daniel Wass,32.0,Central Midfield,Atlético Madrid,2.70m,out,Winter,2.70,Primera Division,...,64,75,79,78,77,11,9,10,13,13
2,Atlético de Madrid,Kieran Trippier,31.0,Right-Back,Newcastle,30.00m,out,Winter,30.00,Primera Division,...,66,77,77,83,81,11,14,8,11,10
3,Newcastle United,Kieran Trippier,31.0,Right-Back,Atlético Madrid,30.00m,in,Winter,30.00,Premier League,...,66,77,77,83,81,11,14,8,11,10
4,FC Barcelona,Ferran Torres,21.0,Left Winger,Man City,55.00m,in,Winter,55.00,Primera Division,...,66,82,35,24,28,12,9,11,15,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,ACF Fiorentina,Matija Nastasi?,28.0,Centre-Back,FC Schalke 04,500Th.,in,Summer,0.50,Serie A,...,52,80,80,78,81,11,16,11,15,13
381,ACF Fiorentina,Julián Illanes,24.0,Centre-Back,Pescara,350Th.,out,Summer,0.35,Serie A,...,40,43,61,66,71,7,7,5,10,15
382,Spezia Calcio,Ardian Ismajli,24.0,Centre-Back,FC Empoli,2.00m,out,Summer,2.00,Serie A,...,31,62,71,72,70,9,14,10,12,13
383,FC Empoli,Ardian Ismajli,24.0,Centre-Back,Spezia Calcio,2.00m,in,Summer,2.00,Serie A,...,31,62,71,72,70,9,14,10,12,13


In [ ]:
result_2021.to_csv('result_2021.csv',encoding='utf-8-sig')

In [ ]:
left = df_2022
right = df22
right.rename({ 'cleaned_name': 'short_name', 'age':'age'}, axis=1, inplace=True)
result_2022=pd.merge(left, right, how="inner", on=['short_name'])

In [ ]:
result_2022

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name_x,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,FC Barcelona,Memphis Depay,28.0,Centre-Forward,Atlético Madrid,3.00m,out,Winter,3.0,Primera Division,...,76,80,23,31,20,8,14,6,12,10
1,Atlético de Madrid,Memphis Depay,28.0,Centre-Forward,Barcelona,3.00m,in,Winter,3.0,Primera Division,...,76,80,23,31,20,8,14,6,12,10
2,FC Barcelona,Héctor Bellerín,27.0,Right-Back,Sporting CP,1.00m,out,Winter,1.0,Primera Division,...,62,70,74,74,76,8,14,14,13,12
3,Atlético de Madrid,Felipe,33.0,Centre-Back,Nottm Forest,2.30m,out,Winter,2.3,Primera Division,...,47,75,86,85,82,9,11,14,9,7
4,Nottingham Forest,Felipe,33.0,Centre-Back,Atlético Madrid,2.30m,in,Winter,2.3,Premier League,...,47,75,86,85,82,9,11,14,9,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,SS Lazio,Denis Vavro,26.0,Centre-Back,FC Copenhagen,4.50m,out,Summer,4.5,Serie A,...,52,67,70,70,69,13,12,14,12,13
493,Atalanta BC,Matteo Lovato,22.0,Centre-Back,Salernitana,7.00m,out,Summer,7.0,Serie A,...,45,53,75,76,73,12,10,6,6,13
494,US Salernitana 1919,Matteo Lovato,22.0,Centre-Back,Atalanta BC,7.00m,in,Summer,7.0,Serie A,...,45,53,75,76,73,12,10,6,6,13
495,US Salernitana 1919,Dylan Bronn,27.0,Centre-Back,FC Metz,1.00m,in,Summer,1.0,Serie A,...,63,71,76,76,76,6,10,9,7,9


In [ ]:
column_names = result_2022.columns
print(column_names)

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name_x', 'year', 'season', 'country', 'short_name', 'Id',
       'Name', 'Team', 'Nationality', 'Position', 'Overall', 'Potential',
       'Value', 'Wage', 'Age', 'league_name_y', 'preferred_foot', 'Work rate',
       'Body type', 'Release cluase', 'Crossing', 'Finishing',
       'Heading accuracy', 'Short passing', 'Volleys', 'Dribbling', 'Curve',
       'Fk accuracy', 'Long passing', 'Ball control', 'Acceleration',
       'Sprint speed', 'Agility', 'Reactions', 'Balance', 'Shot power',
       'Jumping', 'Stamina', 'Strength', 'Long shots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Defensive awareness', 'Standing tackle', 'Sliding tackle', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes'],
      dtype='object')


In [ ]:
result_2022 = result_2022.drop(columns=['Age'])
result_2022 = result_2022.drop(columns=['league_name_y'])
result_2022.rename(columns={'league_name_x': 'league_name'}, inplace=True)
result_2022.rename(columns={'Work rate': 'Work Rate'}, inplace=True)
result_2022.rename(columns={'Body type': 'Body Type'}, inplace=True)
result_2022.rename(columns={'Release cluase': 'Release Clause'}, inplace=True)

In [ ]:
result_2022

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,FC Barcelona,Memphis Depay,28.0,Centre-Forward,Atlético Madrid,3.00m,out,Winter,3.0,Primera Division,...,76,80,23,31,20,8,14,6,12,10
1,Atlético de Madrid,Memphis Depay,28.0,Centre-Forward,Barcelona,3.00m,in,Winter,3.0,Primera Division,...,76,80,23,31,20,8,14,6,12,10
2,FC Barcelona,Héctor Bellerín,27.0,Right-Back,Sporting CP,1.00m,out,Winter,1.0,Primera Division,...,62,70,74,74,76,8,14,14,13,12
3,Atlético de Madrid,Felipe,33.0,Centre-Back,Nottm Forest,2.30m,out,Winter,2.3,Primera Division,...,47,75,86,85,82,9,11,14,9,7
4,Nottingham Forest,Felipe,33.0,Centre-Back,Atlético Madrid,2.30m,in,Winter,2.3,Premier League,...,47,75,86,85,82,9,11,14,9,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,SS Lazio,Denis Vavro,26.0,Centre-Back,FC Copenhagen,4.50m,out,Summer,4.5,Serie A,...,52,67,70,70,69,13,12,14,12,13
493,Atalanta BC,Matteo Lovato,22.0,Centre-Back,Salernitana,7.00m,out,Summer,7.0,Serie A,...,45,53,75,76,73,12,10,6,6,13
494,US Salernitana 1919,Matteo Lovato,22.0,Centre-Back,Atalanta BC,7.00m,in,Summer,7.0,Serie A,...,45,53,75,76,73,12,10,6,6,13
495,US Salernitana 1919,Dylan Bronn,27.0,Centre-Back,FC Metz,1.00m,in,Summer,1.0,Serie A,...,63,71,76,76,76,6,10,9,7,9


In [ ]:
result_2022.to_csv('result_2022.csv',encoding='utf-8-sig')

In [ ]:
df_test_batch = pd.concat([result_2016,result_2017,result_2018,result_2019,result_2020,result_2021,result_2022])

In [ ]:
df_test_batch

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Release Clause
0,Sevilla FC,Timothée Kolodziejczak,25.0,Centre-Back,Bor. M'gladbach,7.50m,out,Winter,7.5,Primera Division,...,NaN,71,77,76,14,6,10,16,7,NaN
1,Borussia Mönchengladbach,Timothée Kolodziejczak,25.0,Centre-Back,Sevilla FC,7.50m,in,Winter,7.5,1 Bundesliga,...,NaN,71,77,76,14,6,10,16,7,NaN
2,Sevilla FC,Hiroshi Kiyotake,27.0,Left Midfield,Cerezo Osaka,5.10m,out,Winter,5.1,Primera Division,...,NaN,23,24,27,15,10,12,7,10,NaN
3,Sevilla FC,Hiroshi Kiyotake,26.0,Left Midfield,Hannover 96,6.50m,in,Summer,6.5,Primera Division,...,NaN,23,24,27,15,10,12,7,10,NaN
4,Real Sociedad,Gerónimo Rulli,24.0,Goalkeeper,Man City,7.00m,in,Winter,7.0,Primera Division,...,NaN,16,13,17,82,77,70,78,83,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,SS Lazio,Denis Vavro,26.0,Centre-Back,FC Copenhagen,4.50m,out,Summer,4.5,Serie A,...,67.0,70,70,69,13,12,14,12,13,6500000.0
493,Atalanta BC,Matteo Lovato,22.0,Centre-Back,Salernitana,7.00m,out,Summer,7.0,Serie A,...,53.0,75,76,73,12,10,6,6,13,9300000.0
494,US Salernitana 1919,Matteo Lovato,22.0,Centre-Back,Atalanta BC,7.00m,in,Summer,7.0,Serie A,...,53.0,75,76,73,12,10,6,6,13,9300000.0
495,US Salernitana 1919,Dylan Bronn,27.0,Centre-Back,FC Metz,1.00m,in,Summer,1.0,Serie A,...,71.0,76,76,76,6,10,9,7,9,13500000.0


In [ ]:
column_names = df_test_batch.columns
print(column_names)

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Nationality', 'Position', 'Overall', 'Potential', 'Value',
       'Wage', 'preferred_foot', 'Work Rate', 'Body Type', 'Crossing',
       'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'Release Clause'],
      dtype='object')


In [ ]:
df_test_batch.to_csv('df_test_batch.csv',encoding='utf-8-sig')#นำข้อมูลออกไปจัดการด้วย power query

In [ ]:
b1='/content/drive/MyDrive/314Final/code model/batch_1.csv'
result_batch = pd.read_csv(b1, encoding=encoding) #จัดการข้อมูล df_test_batchในpower query

In [ ]:
result_batch

,Column1,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,0,Atlético de Madrid,Daniel Wass,32.0,Central Midfield,Valencia,2.70m,in,Winter,2.7,...,78,64,79,78,77,11,9,10,13,13
1,1,Valencia CF,Daniel Wass,32.0,Central Midfield,Atlético Madrid,2.70m,out,Winter,2.7,...,78,64,79,78,77,11,9,10,13,13
2,2,Atlético de Madrid,Kieran Trippier,31.0,Right-Back,Newcastle,30.00m,out,Winter,30.0,...,80,66,77,83,81,11,14,8,11,10
3,3,Newcastle United,Kieran Trippier,31.0,Right-Back,Atlético Madrid,30.00m,in,Winter,30.0,...,80,66,77,83,81,11,14,8,11,10
4,4,FC Barcelona,Ferran Torres,21.0,Left Winger,Man City,55.00m,in,Winter,55.0,...,86,66,35,24,28,12,9,11,15,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3465,438,SS Lazio,Ivan Provedel,28.0,Goalkeeper,Spezia Calcio,2.00m,in,Summer,2.0,...,52,22,22,22,21,75,73,66,72,75
3466,439,Spezia Calcio,Ivan Provedel,28.0,Goalkeeper,Lazio,2.00m,out,Summer,2.0,...,52,22,22,22,21,75,73,66,72,75
3467,442,FC Empoli,Guglielmo Vicario,25.0,Goalkeeper,Cagliari Calcio,8.50m,in,Summer,8.5,...,52,13,23,11,13,70,69,64,73,73
3468,464,Atalanta BC,Éderson,22.0,Central Midfield,Salernitana,21.00m,in,Summer,21.0,...,70,51,29,15,8,87,82,93,88,88


In [ ]:
for f in result_batch['fee_cleaned']:
  if (f == 0.00 ) :
    result_batch.drop(index=result_batch[result_batch['fee_cleaned'] == f].index, inplace=True)

for f in result_batch['fee']:
  if ('loan' in f.lower()) :
    result_batch.drop(index=result_batch[result_batch['fee'] == f].index, inplace=True)


In [ ]:
result_batch

,Column1,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,0,Atlético de Madrid,Daniel Wass,32.0,Central Midfield,Valencia,2.70m,in,Winter,2.7,...,78,64,79,78,77,11,9,10,13,13
1,1,Valencia CF,Daniel Wass,32.0,Central Midfield,Atlético Madrid,2.70m,out,Winter,2.7,...,78,64,79,78,77,11,9,10,13,13
2,2,Atlético de Madrid,Kieran Trippier,31.0,Right-Back,Newcastle,30.00m,out,Winter,30.0,...,80,66,77,83,81,11,14,8,11,10
3,3,Newcastle United,Kieran Trippier,31.0,Right-Back,Atlético Madrid,30.00m,in,Winter,30.0,...,80,66,77,83,81,11,14,8,11,10
4,4,FC Barcelona,Ferran Torres,21.0,Left Winger,Man City,55.00m,in,Winter,55.0,...,86,66,35,24,28,12,9,11,15,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3465,438,SS Lazio,Ivan Provedel,28.0,Goalkeeper,Spezia Calcio,2.00m,in,Summer,2.0,...,52,22,22,22,21,75,73,66,72,75
3466,439,Spezia Calcio,Ivan Provedel,28.0,Goalkeeper,Lazio,2.00m,out,Summer,2.0,...,52,22,22,22,21,75,73,66,72,75
3467,442,FC Empoli,Guglielmo Vicario,25.0,Goalkeeper,Cagliari Calcio,8.50m,in,Summer,8.5,...,52,13,23,11,13,70,69,64,73,73
3468,464,Atalanta BC,Éderson,22.0,Central Midfield,Salernitana,21.00m,in,Summer,21.0,...,70,51,29,15,8,87,82,93,88,88


In [ ]:
#Attribute
df_att = pd.DataFrame(np.array([["Acceleration","Physical"], ["Aggression","Mental"], ["Agility","Physical"], ["Balance","Physical"], ["BallControl","Skills"], ["Composure","Mental"], ["Crossing","Skills"], ["Curve","Skills"], ["Dribbling","Skills"], ["Finishing","Skills"], ["FKAccuracy","Skills"], ["HeadingAccuracy","Skills"], ["Interceptions","Mental"], ["Jumping","Physical"], ["LongPassing","Skills"], ["LongShots","Skills"], ["Best Overall Rating","Skills"], ["Penalties","Skills"], ["Positioning","Mental"], ["Reactions","Physical"], ["ShortPassing","Skills"], ["ShotPower","Skills"], ["SlidingTackle","Skills"], ["SprintSpeed","Physical"], ["Stamina","Physical"], ["StandingTackle","Skills"], ["Strength","Physical"], ["Vision","Mental"], ["Volleys","Skills"], ["GKDiving","Skills"], ["GKHandling","Skills"], ["GKKicking","Skills"], ["GKPositioning","Mental"], ["GKReflexes","Physical"],]), columns=['Attribute', 'Type'])

In [ ]:
df_att

,Attribute,Type
0,Acceleration,Physical
1,Aggression,Mental
2,Agility,Physical
3,Balance,Physical
4,BallControl,Skills
5,Composure,Mental
6,Crossing,Skills
7,Curve,Skills
8,Dribbling,Skills
9,Finishing,Skills


In [ ]:
#ฟังก์ชันหาค่าเยอะสุด n ตัวใน column
>>> def give_largest(col,n):
...     largest = col.nlargest(n).reset_index(drop = True)
...     data = [x for x in largest]
...     index = [f'{i}_largest' for i in range(1,len(largest)+1)]
...     return pd.Series(data,index=index)
...
...
>>> def n_largest(df, axis, n):
...     '''
...     Function to return the n-largest value of each
...     column/row of the input DataFrame.
...     '''
...     return df.apply(give_largest, axis = axis, n = n)

In [ ]:
#ผลรวมค่า attribute แต่ละประเภท ของแต่ละคน
result_batch['Mental'] = result_batch[["Aggression","Interceptions","Positioning","Vision","GK Positioning"]].sum(axis=1)
result_batch['Physical'] = result_batch[["Acceleration","Agility","Balance","Jumping","Reactions","Sprint speed","Stamina","Strength","GK Reflexes"]].sum(axis=1)
result_batch['Skills'] = result_batch[["Ball control", "Crossing", "Curve", "Dribbling", "Finishing", "Fk accuracy", "Heading accuracy", "Long passing", "Long shots", "Penalties", "Short passing", "Shot power", "Sliding tackle", "Standing tackle", "Volleys", "GK Diving", "GK Handling", "GK Kicking"]].sum(axis=1)


In [ ]:
#ผลรวมค่า attribute แต่ละประเภท ของแต่ละคน
result_batch['Mental_top3'] = n_largest(result_batch[["Aggression","Interceptions","Positioning","Vision","GK Positioning"]],axis = 1, n = 3).sum(axis =1)
result_batch['Physical_top3'] = n_largest(result_batch[["Acceleration","Agility","Balance","Jumping","Reactions","Sprint speed","Stamina","Strength","GK Reflexes"]],axis = 1, n = 3).sum(axis =1)
result_batch['Skills_top3'] = n_largest(result_batch[["Ball control", "Crossing", "Curve", "Dribbling", "Finishing", "Fk accuracy", "Heading accuracy", "Long passing", "Long shots", "Penalties", "Short passing", "Shot power", "Sliding tackle", "Standing tackle", "Volleys", "GK Diving", "GK Handling", "GK Kicking"]],axis = 1, n = 3).sum(axis =1)


In [ ]:
result_batch

,Column1,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,...,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Mental,Physical,Skills,Mental_top3,Physical_top3,Skills_top3
0,0,Atlético de Madrid,Daniel Wass,32.0,Central Midfield,Valencia,2.70m,in,Winter,2.7,...,9,10,13,13,318,614,1188,231,242,250
1,1,Valencia CF,Daniel Wass,32.0,Central Midfield,Atlético Madrid,2.70m,out,Winter,2.7,...,9,10,13,13,318,614,1188,231,242,250
2,2,Atlético de Madrid,Kieran Trippier,31.0,Right-Back,Newcastle,30.00m,out,Winter,30.0,...,14,8,11,10,313,615,1158,232,246,255
3,3,Newcastle United,Kieran Trippier,31.0,Right-Back,Atlético Madrid,30.00m,in,Winter,30.0,...,14,8,11,10,313,615,1158,232,246,255
4,4,FC Barcelona,Ferran Torres,21.0,Left Winger,Man City,55.00m,in,Winter,55.0,...,9,11,15,12,257,615,1035,208,245,241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3465,438,SS Lazio,Ivan Provedel,28.0,Goalkeeper,Spezia Calcio,2.00m,in,Summer,2.0,...,73,66,72,75,189,451,530,151,201,214
3466,439,Spezia Calcio,Ivan Provedel,28.0,Goalkeeper,Lazio,2.00m,out,Summer,2.0,...,73,66,72,75,189,451,530,151,201,214
3467,442,FC Empoli,Guglielmo Vicario,25.0,Goalkeeper,Cagliari Calcio,8.50m,in,Summer,8.5,...,69,64,73,73,165,368,432,142,187,203
3468,464,Atalanta BC,Éderson,22.0,Central Midfield,Salernitana,21.00m,in,Summer,21.0,...,82,93,88,88,243,596,715,196,254,262


In [ ]:
result_batch.reset_index(drop=True, inplace=True)

In [ ]:
batch = result_batch.drop(['club_name',
                              'club_involved_name',
                              'Nationality',
                              'transfer_movement',

                              'short_name',
                              'player_name',
                               'Name',
                              'position',
                              'fee',
                              'transfer_period',
                              'league_name',
                              'season',
                              'country',



                              'preferred_foot',
                              'Body Type',




                              'Work Rate',

                              'Position.1',


                              'Team',
                              'year',
                              'Mental',
                              'Skills',
                              'Physical',
                              'Mental_top3',
                              'Physical_top3',
                              'Skills_top3',
                               'Column1'

                              ], axis=1)
batch.describe()

,age,fee_cleaned,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
count,3468.000000,3470.000000,3470.000000,3470.000000,3.470000e+03,3470.000000,3470.000000,3470.000000,3470.000000,3470.000000,...,3470.000000,3470.000000,3470.000000,3470.000000,3470.000000,3470.000000,3470.000000,3470.000000,3470.000000,3470.000000
mean,25.451269,12.282538,74.826513,79.873487,1.060916e+07,42181.772334,58.924496,54.654179,60.513545,68.740058,...,62.004899,55.456484,52.051873,54.589914,51.697118,14.627666,14.375504,14.161960,14.446686,14.653026
std,3.587578,16.842402,5.639490,4.517908,1.326999e+07,43260.718148,17.969484,20.481076,16.856001,12.730974,...,14.277052,15.612294,22.787592,23.958968,24.250665,16.448886,15.726050,14.912341,16.011091,16.899907
min,17.000000,0.001000,52.000000,62.000000,9.000000e+04,500.000000,7.000000,5.000000,8.000000,11.000000,...,10.000000,11.000000,5.000000,8.000000,8.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,23.000000,2.500000,72.000000,77.000000,2.900000e+06,16000.000000,49.000000,39.000000,53.000000,65.000000,...,54.000000,45.000000,30.000000,31.000000,26.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,25.000000,6.500000,75.000000,80.000000,6.500000e+06,30000.000000,64.000000,59.000000,64.000000,72.000000,...,65.000000,57.000000,59.000000,65.000000,61.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,28.000000,15.400000,78.000000,83.000000,1.250000e+07,50000.000000,72.000000,71.000000,72.000000,77.000000,...,73.000000,67.000000,72.000000,76.000000,73.000000,14.000000,14.000000,14.000000,14.000000,14.000000
max,36.000000,222.000000,94.000000,95.000000,1.375000e+08,575000.000000,91.000000,95.000000,93.000000,92.000000,...,91.000000,96.000000,90.000000,92.000000,90.000000,90.000000,91.000000,93.000000,88.000000,90.000000


In [ ]:
batch

,age,fee_cleaned,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,32.0,2.7,80,80,15500000,38000,79,75,69,80,...,78,64,79,78,77,11,9,10,13,13
1,32.0,2.7,80,80,15500000,38000,79,75,69,80,...,78,64,79,78,77,11,9,10,13,13
2,31.0,30.0,83,83,32000000,71000,86,48,74,80,...,80,66,77,83,81,11,14,8,11,10
3,31.0,30.0,83,83,32000000,71000,86,48,74,80,...,80,66,77,83,81,11,14,8,11,10
4,21.0,55.0,81,89,56500000,95000,77,77,70,80,...,86,66,35,24,28,12,9,11,15,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3465,28.0,2.0,74,76,3900000,12000,21,17,17,29,...,52,22,22,22,21,75,73,66,72,75
3466,28.0,2.0,74,76,3900000,12000,21,17,17,29,...,52,22,22,22,21,75,73,66,72,75
3467,25.0,8.5,71,77,2400000,12000,11,6,11,27,...,52,13,23,11,13,70,69,64,73,73
3468,22.0,21.0,89,91,94000000,200000,20,14,14,61,...,70,51,29,15,8,87,82,93,88,88


In [ ]:
batch_full = batch

In [ ]:
batch_full = batch_full.dropna(how = 'any')

In [ ]:
batch_full.dropna(how = 'any').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3468 entries, 0 to 3469
Data columns (total 39 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  3468 non-null   float64
 1   fee_cleaned          3468 non-null   float64
 2   Overall              3468 non-null   int64  
 3   Potential            3468 non-null   int64  
 4   Value                3468 non-null   int64  
 5   Wage                 3468 non-null   int64  
 6   Crossing             3468 non-null   int64  
 7   Finishing            3468 non-null   int64  
 8   Heading accuracy     3468 non-null   int64  
 9   Short passing        3468 non-null   int64  
 10  Volleys              3468 non-null   int64  
 11  Dribbling            3468 non-null   int64  
 12  Curve                3468 non-null   int64  
 13  Fk accuracy          3468 non-null   int64  
 14  Long passing         3468 non-null   int64  
 15  Ball control         3468 non-null   i

# ขั้นตอนการเริ่มทำ Machine learning

In [ ]:
y = batch_full['fee_cleaned']#label
x = batch_full.drop('fee_cleaned',axis=1)

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3468 entries, 0 to 3469
Data columns (total 38 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  3468 non-null   float64
 1   Overall              3468 non-null   int64  
 2   Potential            3468 non-null   int64  
 3   Value                3468 non-null   int64  
 4   Wage                 3468 non-null   int64  
 5   Crossing             3468 non-null   int64  
 6   Finishing            3468 non-null   int64  
 7   Heading accuracy     3468 non-null   int64  
 8   Short passing        3468 non-null   int64  
 9   Volleys              3468 non-null   int64  
 10  Dribbling            3468 non-null   int64  
 11  Curve                3468 non-null   int64  
 12  Fk accuracy          3468 non-null   int64  
 13  Long passing         3468 non-null   int64  
 14  Ball control         3468 non-null   int64  
 15  Acceleration         3468 non-null   i

In [ ]:
y

0        2.7
1        2.7
2       30.0
3       30.0
4       55.0
        ... 
3465     2.0
3466     2.0
3467     8.5
3468    21.0
3469    21.0
Name: fee_cleaned, Length: 3468, dtype: float64

In [ ]:
scale = StandardScaler()

In [ ]:
test = scale.fit_transform(x)

test.shape

(3468, 38)

In [ ]:
test

array([[ 1.82565402,  0.91764389,  0.02801467, ..., -0.27935028,
        -0.09054139, -0.09806554],
       [ 1.82565402,  0.91764389,  0.02801467, ..., -0.27935028,
        -0.09054139, -0.09806554],
       [ 1.54687421,  1.44957876,  0.69194329, ..., -0.41345697,
        -0.21544096, -0.27556622],
       ...,
       [-0.12580461, -0.67816072, -0.63591394, ...,  3.3415305 ,
         3.65644582,  3.45194798],
       [-0.96214402,  2.5134485 ,  2.4624196 , ...,  5.28607758,
         4.59319263,  4.33945136],
       [-0.96214402,  2.5134485 ,  2.4624196 , ...,  5.28607758,
         4.59319263,  4.33945136]])

In [ ]:
pca = PCA(n_components = 0.99)

In [ ]:
list(x)

['age',
 'Overall',
 'Potential',
 'Value',
 'Wage',
 'Crossing',
 'Finishing',
 'Heading accuracy',
 'Short passing',
 'Volleys',
 'Dribbling',
 'Curve',
 'Fk accuracy',
 'Long passing',
 'Ball control',
 'Acceleration',
 'Sprint speed',
 'Agility',
 'Reactions',
 'Balance',
 'Shot power',
 'Jumping',
 'Stamina',
 'Strength',
 'Long shots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Defensive awareness',
 'Standing tackle',
 'Sliding tackle',
 'GK Diving',
 'GK Handling',
 'GK Kicking',
 'GK Positioning',
 'GK Reflexes']

In [ ]:
name = ['const','age',
 'Overall',
 'Potential',
 'Value',
 'Wage',
 'Crossing',
 'Finishing',
 'Heading accuracy',
 'Short passing',
 'Volleys',
 'Dribbling',
 'Curve',
 'Fk accuracy',
 'Long passing',
 'Ball control',
 'Acceleration',
 'Sprint speed',
 'Agility',
 'Reactions',
 'Balance',
 'Shot power',
 'Jumping',
 'Stamina',
 'Strength',
 'Long shots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Defensive awareness',
 'Standing tackle',
 'Sliding tackle',
 'GK Diving',
 'GK Handling',
 'GK Kicking',
 'GK Positioning',
 'GK Reflexes']

In [ ]:
x_pca = pca.fit_transform(test)

In [ ]:
x_pca

array([[-4.17611   ,  0.77129301,  1.87285655, ...,  0.4189123 ,
         0.02320694,  0.1716361 ],
       [-4.17611   ,  0.77129301,  1.87285655, ...,  0.4189123 ,
         0.02320694,  0.1716361 ],
       [-4.06874201,  1.08300845,  2.87611899, ..., -0.33891486,
         0.10511157,  0.24978314],
       ...,
       [14.92668467, -2.81411201,  0.84256084, ...,  0.19043202,
        -0.16447545, -0.21013019],
       [ 9.00763223, -4.78174576,  9.94099935, ...,  0.06146235,
         0.06276322,  0.39189345],
       [ 9.00763223, -4.78174576,  9.94099935, ...,  0.06146235,
         0.06276322,  0.39189345]])

In [ ]:
pca.n_components_

29

In [ ]:
pca.explained_variance_ratio_

array([0.43953617, 0.16677915, 0.1135715 , 0.0593923 , 0.04365239,
       0.0318046 , 0.01831933, 0.01501149, 0.01136238, 0.00935808,
       0.00929426, 0.00805179, 0.00689943, 0.00633858, 0.00548959,
       0.00545402, 0.00524262, 0.00474309, 0.00425879, 0.00369635,
       0.00360707, 0.0033382 , 0.00299391, 0.00263753, 0.00247187,
       0.00214048, 0.00198887, 0.00178776, 0.00177048])

In [ ]:
import statsmodels
import statsmodels.api as sm

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    test, y, test_size=0.2, random_state=13
)

# OLS Regression Model

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats


X2 = sm.add_constant(X_train)
est = sm.OLS(list(y_train ), X2)
results = est.fit_regularized(alpha=0.01, L1_wt=0.1, refit=True)
print(results.summary(xname = name))


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.412
Model:                            OLS   Adj. R-squared:                  0.404
Method:                 Least Squares   F-statistic:                     50.38
Date:                Mon, 18 Dec 2023   Prob (F-statistic):          4.03e-282
Time:                        05:41:14   Log-Likelihood:                -11004.
No. Observations:                2774   AIC:                         2.209e+04
Df Residuals:                    2736   BIC:                         2.232e+04
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  12.3739    

# find best alpha for lasso

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import permutation_importance


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

# Define the parameter grid
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Lasso Regression with Grid Search
lasso_model = Lasso()
grid_search = GridSearchCV(lasso_model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

print('\nLasso Regression with Grid Search:')
print('Best alpha: {}'.format(grid_search.best_params_['alpha']))
print('Training score: {}'.format(grid_search.best_estimator_.score(X_train, y_train)))
print('Test score: {}'.format(grid_search.best_estimator_.score(X_test, y_test)))

y_pred_lasso = grid_search.best_estimator_.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
rmse_lasso = math.sqrt(mse_lasso)
print('RMSE: {}'.format(rmse_lasso))



Lasso Regression with Grid Search:
Best alpha: 0.1
Training score: 0.4044990951559665
Test score: 0.5032224471157296
RMSE: 12.355804807490303


# GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso
# Assuming X_train, X_test, y_train, and y_test are already defined

# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

print('Linear Regression:')
print('Training score: {}'.format(lr_model.score(X_train, y_train)))
print('Test score: {}'.format(lr_model.score(X_test, y_test)))

y_pred_lr = lr_model.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = math.sqrt(mse_lr)
print('RMSE: {}'.format(rmse_lr))

# Lasso Regression
alpha = 0.1
lasso_model = Lasso(alpha=alpha)
lasso_model.fit(X_train, y_train)

print('\nLasso Regression (alpha={}):'.format(alpha))
print('Training score: {}'.format(lasso_model.score(X_train, y_train)))
print('Test score: {}'.format(lasso_model.score(X_test, y_test)))

y_pred_lasso = lasso_model.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
rmse_lasso = math.sqrt(mse_lasso)
print('RMSE: {}'.format(rmse_lasso))

# Gradient Boosting Regression
gb_model = GradientBoostingRegressor()
gb_model.fit(X_train, y_train)

print('\nGradient Boosting Regression:')
print('Training score: {}'.format(gb_model.score(X_train, y_train)))
print('Test score: {}'.format(gb_model.score(X_test, y_test)))

y_pred_gb = gb_model.predict(X_test)
mse_gb = mean_squared_error(y_test, y_pred_gb)
rmse_gb = math.sqrt(mse_gb)
print('RMSE: {}'.format(rmse_gb))


Linear Regression:
Training score: 0.41169587892634174
Test score: 0.4964593894366547
RMSE: 12.439625559841009

Lasso Regression (alpha=0.1):
Training score: 0.4044990951559665
Test score: 0.5032224471157296
RMSE: 12.355804807490303

Gradient Boosting Regression:
Training score: 0.7579770755157998
Test score: 0.6875594083736385
RMSE: 9.798820062112508


# Random Forest(runtime 1hr+)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Assuming X_train, X_test, y_train, and y_test are already defined

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Random Forest Regression with Grid Search
rf_model = RandomForestRegressor()
grid_search_rf = GridSearchCV(rf_model, param_grid, cv=5)  # You can adjust the number of folds in cross-validation (cv)
grid_search_rf.fit(X_train, y_train)

print('\nRandom Forest Regression with Grid Search:')
print('Best parameters: {}'.format(grid_search_rf.best_params_))
print('Training score: {}'.format(grid_search_rf.best_estimator_.score(X_train, y_train)))
print('Test score: {}'.format(grid_search_rf.best_estimator_.score(X_test, y_test)))

y_pred_rf = grid_search_rf.best_estimator_.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = math.sqrt(mse_rf)
print('RMSE: {}'.format(rmse_rf))


Random Forest Regression with Grid Search:
Best parameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Training score: 0.9659755913834501
Test score: 0.8103826850551531
RMSE: 7.633593918073204


# มาทำนายแยกแต่ละปีเพื่อนำข้อมูลไปเทียบ

ทำนายราคานักเตะในปี2016

In [ ]:
batch2016 = result_2016.drop(['club_name',
                              'club_involved_name',
                              'Nationality',
                              'transfer_movement',
                              'Id',
                              'Position',
                              'fee_cleaned',

                              'short_name',
                              'player_name',
                               'Name',
                              'position',
                              'fee',
                              'transfer_period',
                              'league_name',
                              'season',
                              'country',



                              'preferred_foot',
                              'Body Type',



                              'Composure',
                              'Work Rate',




                              'Team',
                              'year',


                              ], axis=1)
batch2016.describe()

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
count,539.000000,539.000000,539.000000,5.390000e+02,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,...,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000,539.000000
mean,25.478664,74.627087,79.077922,7.177338e+06,51215.213358,60.716141,54.560297,61.267161,69.760668,53.658627,...,62.810761,57.380334,50.480519,55.287570,52.521336,13.808905,13.489796,13.452690,13.337662,13.421150
std,3.345225,4.939042,4.275965,7.666839e+06,37614.732423,16.427580,19.589789,15.791599,11.651709,18.292704,...,13.913881,14.964955,24.310589,24.017668,24.372440,14.645585,13.842629,13.029959,14.219897,15.252363
min,18.000000,57.000000,66.000000,1.700000e+05,2000.000000,11.000000,10.000000,10.000000,22.000000,10.000000,...,10.000000,12.000000,11.000000,10.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,23.000000,72.000000,76.000000,2.450000e+06,22500.000000,52.000000,40.000000,54.000000,66.000000,41.000000,...,54.500000,48.000000,25.000000,30.500000,27.000000,8.000000,8.000000,8.500000,8.000000,8.000000
50%,25.000000,75.000000,79.000000,4.500000e+06,45000.000000,66.000000,58.000000,64.000000,72.000000,56.000000,...,66.000000,58.000000,56.000000,65.000000,61.000000,11.000000,11.000000,11.000000,10.000000,10.000000
75%,28.000000,78.000000,82.000000,9.000000e+06,70000.000000,72.000000,71.000000,72.000000,77.000000,68.000000,...,73.000000,70.000000,74.000000,77.000000,75.000000,14.000000,14.000000,14.000000,14.000000,14.000000
max,34.000000,87.000000,93.000000,6.250000e+07,220000.000000,90.000000,89.000000,91.000000,86.000000,91.000000,...,87.000000,96.000000,86.000000,87.000000,87.000000,83.000000,82.000000,87.000000,87.000000,86.000000


In [ ]:
batch2016.columns

Index(['age', 'Overall', 'Potential', 'Value', 'Wage', 'Crossing', 'Finishing',
       'Heading accuracy', 'Short passing', 'Volleys', 'Dribbling', 'Curve',
       'Fk accuracy', 'Long passing', 'Ball control', 'Acceleration',
       'Sprint speed', 'Agility', 'Reactions', 'Balance', 'Shot power',
       'Jumping', 'Stamina', 'Strength', 'Long shots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Defensive awareness', 'Standing tackle', 'Sliding tackle', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes'],
      dtype='object')

In [ ]:
batch2016

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,25.0,74,80,4000000,45000,70,22,67,73,26,...,58,49,71,77,76,14,6,10,16,7
1,25.0,74,80,4000000,45000,70,22,67,73,26,...,58,49,71,77,76,14,6,10,16,7
2,27.0,76,78,6000000,50000,83,62,35,80,69,...,80,66,23,24,27,15,10,12,7,10
3,26.0,76,78,6000000,50000,83,62,35,80,69,...,80,66,23,24,27,15,10,12,7,10
4,24.0,79,86,12500000,60000,12,15,12,32,16,...,49,24,16,13,17,82,77,70,78,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,21.0,70,80,1900000,20000,72,59,44,70,44,...,59,60,65,69,72,13,14,8,11,13
535,31.0,71,71,1200000,20000,44,28,70,54,20,...,29,38,74,74,72,8,9,14,15,6
536,31.0,71,71,1200000,20000,44,28,70,54,20,...,29,38,74,74,72,8,9,14,15,6
537,23.0,76,85,7000000,70000,60,27,76,70,34,...,57,40,77,79,74,14,15,10,12,8


In [ ]:
test2016 = scale.fit_transform(batch2016)

test2016.shape

(539, 38)

In [ ]:
batch_pca2016 = pca.fit_transform(test2016)
batch_pca2016

array([[ 0.79476724, -2.97586944, -0.31757502, ...,  0.0262034 ,
        -0.40553079, -0.02268512],
       [ 0.79476724, -2.97586944, -0.31757502, ...,  0.0262034 ,
        -0.40553079, -0.02268512],
       [-2.68530864,  3.79900703, -0.30257517, ...,  0.29974139,
        -0.32194331, -0.27196734],
       ...,
       [ 5.60600667, -4.39152924, -1.22687633, ...,  0.58631753,
        -0.16347908,  0.07142613],
       [ 1.2488019 , -3.03211477,  0.93030684, ...,  0.29697693,
        -0.24005129, -0.41107644],
       [ 6.90070776, -4.46413146, -1.23057466, ..., -0.18629569,
         0.28910115,  0.15703939]])

In [ ]:
y_pred2016 = gb_model.predict(test2016)
y_pred2016

array([  8.22368412,   8.22368412,   6.85282059,   6.85282059,
        21.90231117,  21.90231117,  21.90231117,   8.3946799 ,
         9.724601  ,   8.3946799 ,   9.724601  ,  37.49948488,
        37.49948488,  31.44743411,  31.44743411,  13.96644072,
        13.96644072,   7.19364794,   7.19364794,  16.01858843,
        16.01858843,  23.82313808,  23.82313808,   5.85081294,
        19.70765508,  19.70765508,   7.70129838,   7.70129838,
         2.72005849,  18.60218248,   2.72005849,  18.60218248,
        22.24481952,  22.24481952,  15.72573419,  15.72573419,
         8.73539342,   8.73539342,   9.61896738,   9.61896738,
         5.64578689,   5.64578689,   4.70556137,   4.70556137,
        20.32282132,  20.32282132,   6.63124855,   9.79202463,
        18.37452471,   3.86901292,   3.86901292,  30.86112663,
        15.01394427,  30.86112663,  15.01394427,   3.51698767,
         3.51698767,   4.77740161,  26.00919638,  26.00919638,
         7.43683467,   7.43683467,   5.88950587,   5.88

In [ ]:
fee_pred2016 = pd.DataFrame(y_pred2016)
fee_pred2016 = fee_pred2016.rename(columns={0: 'fee_pred'})

In [ ]:
fee_pred2016

,fee_pred
0,8.223684
1,8.223684
2,6.852821
3,6.852821
4,21.902311
...,...
534,9.699908
535,4.817317
536,4.817317
537,25.170204


In [ ]:
batch2016['fee_pred'] = fee_pred2016
batch2016.columns
batch2016

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_pred
0,25.0,74,80,4000000,45000,70,22,67,73,26,...,49,71,77,76,14,6,10,16,7,8.223684
1,25.0,74,80,4000000,45000,70,22,67,73,26,...,49,71,77,76,14,6,10,16,7,8.223684
2,27.0,76,78,6000000,50000,83,62,35,80,69,...,66,23,24,27,15,10,12,7,10,6.852821
3,26.0,76,78,6000000,50000,83,62,35,80,69,...,66,23,24,27,15,10,12,7,10,6.852821
4,24.0,79,86,12500000,60000,12,15,12,32,16,...,24,16,13,17,82,77,70,78,83,21.902311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,21.0,70,80,1900000,20000,72,59,44,70,44,...,60,65,69,72,13,14,8,11,13,9.699908
535,31.0,71,71,1200000,20000,44,28,70,54,20,...,38,74,74,72,8,9,14,15,6,4.817317
536,31.0,71,71,1200000,20000,44,28,70,54,20,...,38,74,74,72,8,9,14,15,6,4.817317
537,23.0,76,85,7000000,70000,60,27,76,70,34,...,40,77,79,74,14,15,10,12,8,25.170204


In [ ]:
result_2016['fee_pred'] = fee_pred2016
result_2016.columns

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Nationality', 'Position', 'Overall', 'Potential', 'Value',
       'Wage', 'preferred_foot', 'Work Rate', 'Body Type', 'Crossing',
       'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'fee_pred'],
      dtype='object')

In [ ]:
new_column_order2016 = ['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Nationality', 'Position', 'Overall', 'Potential', 'Value',
       'Wage', 'preferred_foot', 'Work Rate', 'Body Type', 'Crossing',
       'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes','fee_cleaned', 'fee_pred']
Final2016 = result_2016[new_column_order2016]

In [ ]:
Final2016

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,league_name,year,...,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_cleaned,fee_pred
0,Sevilla FC,Timothée Kolodziejczak,25.0,Centre-Back,Bor. M'gladbach,7.50m,out,Winter,Primera Division,2016,...,71,77,76,14,6,10,16,7,7.5,8.223684
1,Borussia Mönchengladbach,Timothée Kolodziejczak,25.0,Centre-Back,Sevilla FC,7.50m,in,Winter,1 Bundesliga,2016,...,71,77,76,14,6,10,16,7,7.5,8.223684
2,Sevilla FC,Hiroshi Kiyotake,27.0,Left Midfield,Cerezo Osaka,5.10m,out,Winter,Primera Division,2016,...,23,24,27,15,10,12,7,10,5.1,6.852821
3,Sevilla FC,Hiroshi Kiyotake,26.0,Left Midfield,Hannover 96,6.50m,in,Summer,Primera Division,2016,...,23,24,27,15,10,12,7,10,6.5,6.852821
4,Real Sociedad,Gerónimo Rulli,24.0,Goalkeeper,Man City,7.00m,in,Winter,Primera Division,2016,...,16,13,17,82,77,70,78,83,7.0,21.902311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,SS Lazio,Wallace,21.0,Centre-Back,SC Braga,8.00m,in,Summer,Serie A,2016,...,65,69,72,13,14,8,11,13,8.0,9.699908
535,Delfino Pescara 1936,Andrea Coda,31.0,Centre-Back,Sampdoria,200Th.,in,Summer,Serie A,2016,...,74,74,72,8,9,14,15,6,0.2,4.817317
536,UC Sampdoria,Andrea Coda,31.0,Centre-Back,Pescara,200Th.,out,Summer,Serie A,2016,...,74,74,72,8,9,14,15,6,0.2,4.817317
537,AS Roma,Antonio Rüdiger,23.0,Centre-Back,VfB Stuttgart,9.00m,in,Summer,Serie A,2016,...,77,79,74,14,15,10,12,8,9.0,25.170204


ทำนายราคานักเตะในปี2017

In [ ]:
batch2017 = result_2017.drop(['club_name',
                              'club_involved_name',
                              'Nationality',
                              'transfer_movement',
                              'Id',
                              'Position',
                              'fee_cleaned',

                              'short_name',
                              'player_name',
                               'Name',
                              'position',
                              'fee',
                              'transfer_period',
                              'league_name',
                              'season',
                              'country',



                              'preferred_foot',
                              'Body Type',



                              'Composure',
                              'Work Rate',




                              'Team',
                              'year',


                              ], axis=1)
batch2017.describe()

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
count,584.000000,584.000000,584.000000,5.840000e+02,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,...,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000,584.000000
mean,25.633562,75.414384,79.982877,9.264589e+06,46287.671233,60.532534,54.700342,61.532534,69.184932,53.039384,...,61.087329,56.931507,51.729452,56.095890,53.566781,13.188356,12.717466,12.751712,12.755137,13.000000
std,3.686139,5.871365,4.400575,1.093403e+07,46396.450876,17.065376,19.957354,15.723565,11.857648,18.556025,...,15.041540,14.792751,24.016549,24.001095,24.532032,13.642538,12.759040,12.319573,13.054838,13.827658
min,17.000000,56.000000,65.000000,2.000000e+05,2000.000000,9.000000,8.000000,8.000000,17.000000,9.000000,...,19.000000,11.000000,9.000000,10.000000,11.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,23.000000,72.000000,77.000000,2.600000e+06,20000.000000,52.000000,40.000000,54.000000,65.000000,37.000000,...,52.000000,47.000000,27.000000,34.000000,29.750000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,26.000000,76.000000,80.000000,6.000000e+06,30000.000000,65.000000,59.000000,63.000000,72.000000,57.000000,...,65.000000,58.500000,59.000000,66.000000,63.000000,11.000000,10.000000,10.000000,10.000000,11.000000
75%,28.000000,79.000000,83.000000,1.200000e+07,60000.000000,74.000000,71.000000,73.000000,77.000000,68.000000,...,73.000000,68.250000,74.000000,77.000000,76.000000,13.000000,13.000000,13.000000,13.250000,13.000000
max,36.000000,92.000000,95.000000,1.090000e+08,450000.000000,90.000000,89.000000,93.000000,89.000000,91.000000,...,88.000000,87.000000,87.000000,88.000000,88.000000,88.000000,82.000000,82.000000,81.000000,87.000000


In [ ]:
batch2017

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,33.0,85,85,19500000.0,200000,63,42,68,79,51,...,68,60,84,87,85,6,10,8,5,5
1,29.0,72,72,1600000.0,20000,12,13,12,28,11,...,34,23,16,19,18,70,68,63,73,74
2,29.0,85,86,38500000.0,200000,63,88,82,65,81,...,74,76,28,39,34,11,13,12,8,11
3,29.0,84,85,33000000.0,140000,78,72,58,80,73,...,80,77,41,58,44,13,15,9,12,5
4,29.0,72,72,1600000.0,20000,12,13,12,28,11,...,34,23,16,19,18,70,68,63,73,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,30.0,87,87,34000000.0,180000,44,37,84,75,55,...,74,70,84,88,88,2,2,3,2,4
580,23.0,72,79,2600000.0,10000,27,22,72,55,26,...,19,34,74,73,70,9,12,7,15,15
581,32.0,75,75,2700000.0,20000,34,20,74,62,19,...,46,35,80,80,76,11,6,8,9,5
582,32.0,75,75,2700000.0,20000,34,20,74,62,19,...,46,35,80,80,76,11,6,8,9,5


In [ ]:
test2017 = scale.fit_transform(batch2017)

test2017.shape

(584, 38)

In [ ]:
batch_pca2017 = pca.fit_transform(test2017)
batch_pca2017

array([[-2.29309045, -3.68633455,  3.41780985, ...,  0.13977057,
         0.44859428, -0.03768777],
       [15.33027995,  4.29836144,  3.1253892 , ..., -0.08126864,
        -0.38059924, -0.15028522],
       [-3.22165467,  0.74656663,  4.06136647, ..., -0.05280673,
         0.11107207, -0.23886253],
       ...,
       [ 3.90740788, -3.90277532, -0.61632309, ..., -0.06391106,
        -0.15674937,  0.21913317],
       [ 3.90740788, -3.90277532, -0.61632309, ..., -0.06391106,
        -0.15674937,  0.21913317],
       [ 5.04923715, -3.84182053, -1.13132553, ..., -0.66690168,
        -0.03805535,  0.06191687]])

In [ ]:
y_pred2017 = gb_model.predict(test2017)
y_pred2017

array([ 9.93283901e+00,  1.38760054e+01,  5.09273594e+01,  5.12465029e+01,
        1.38760054e+01,  5.09273594e+01,  5.12465029e+01,  3.86316404e+01,
        2.20723494e+01, -1.24396100e-01,  7.21490321e+00,  3.03899376e-01,
        7.64319869e+00,  1.54684060e+00,  1.54684060e+00,  2.15615522e+01,
        2.19441052e+01,  2.19441052e+01,  1.15633906e+01,  4.39525476e+01,
        4.39525476e+01,  7.40430827e+00,  7.40430827e+00,  2.28243370e+01,
        2.28243370e+01,  1.04918364e+01,  1.60967917e+01,  1.60967917e+01,
        9.12207519e+00,  9.12207519e+00,  1.45063740e+01,  1.45063740e+01,
        2.07881778e+01,  1.19257857e+01,  2.07881778e+01,  1.19257857e+01,
        2.26436982e+01,  2.26436982e+01,  1.38438211e+01,  1.38438211e+01,
        1.15517119e+02,  1.15517119e+02,  2.11865140e+02,  2.11865140e+02,
        1.10852840e+01,  1.10852840e+01,  2.23384375e+01,  2.23384375e+01,
        1.82447460e+01,  1.26139185e+01,  1.26139185e+01,  3.74530853e+01,
        3.74530853e+01,  

In [ ]:
fee_pred2017 = pd.DataFrame(y_pred2017)
fee_pred2017 = fee_pred2017.rename(columns={0: 'fee_pred'})

In [ ]:
fee_pred2017

,fee_pred
0,9.932839
1,13.876005
2,50.927359
3,51.246503
4,13.876005
...,...
579,35.351929
580,7.816261
581,2.053159
582,2.053159


In [ ]:
batch2017['fee_pred'] = fee_pred2017
batch2017.columns
batch2017

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_pred
0,33.0,85,85,19500000.0,200000,63,42,68,79,51,...,60,84,87,85,6,10,8,5,5,9.932839
1,29.0,72,72,1600000.0,20000,12,13,12,28,11,...,23,16,19,18,70,68,63,73,74,13.876005
2,29.0,85,86,38500000.0,200000,63,88,82,65,81,...,76,28,39,34,11,13,12,8,11,50.927359
3,29.0,84,85,33000000.0,140000,78,72,58,80,73,...,77,41,58,44,13,15,9,12,5,51.246503
4,29.0,72,72,1600000.0,20000,12,13,12,28,11,...,23,16,19,18,70,68,63,73,74,13.876005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,30.0,87,87,34000000.0,180000,44,37,84,75,55,...,70,84,88,88,2,2,3,2,4,35.351929
580,23.0,72,79,2600000.0,10000,27,22,72,55,26,...,34,74,73,70,9,12,7,15,15,7.816261
581,32.0,75,75,2700000.0,20000,34,20,74,62,19,...,35,80,80,76,11,6,8,9,5,2.053159
582,32.0,75,75,2700000.0,20000,34,20,74,62,19,...,35,80,80,76,11,6,8,9,5,2.053159


In [ ]:
result_2017['fee_pred'] = fee_pred2017
result_2017.columns

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Position', 'Overall', 'Potential', 'Value', 'Wage',
       'Nationality', 'preferred_foot', 'Work Rate', 'Body Type', 'Crossing',
       'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'fee_pred'],
      dtype='object')

In [ ]:
result_2017

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_pred
0,FC Barcelona,Javier Mascherano,33.0,Defensive Midfield,HB CFFC,5.50m,out,Winter,5.500,Primera Division,...,86,84,87,85,6,10,8,5,5,9.932839
1,Atlético de Madrid,Diego Costa,29.0,Centre-Forward,Chelsea,60.00m,in,Winter,60.000,Primera Division,...,43,16,19,18,70,68,63,73,74,13.876005
2,Atlético de Madrid,Diego Costa,29.0,Centre-Forward,Chelsea,60.00m,in,Winter,60.000,Primera Division,...,84,28,39,34,11,13,12,8,11,50.927359
3,Atlético de Madrid,Diego Costa,29.0,Centre-Forward,Chelsea,60.00m,in,Winter,60.000,Primera Division,...,73,41,58,44,13,15,9,12,5,51.246503
4,Chelsea FC,Diego Costa,29.0,Centre-Forward,Atlético Madrid,60.00m,out,Winter,60.000,Premier League,...,43,16,19,18,70,68,63,73,74,13.876005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,AC Milan,Leonardo Bonucci,30.0,Centre-Back,Juventus,42.00m,in,Summer,42.000,Serie A,...,82,84,88,88,2,2,3,2,4,35.351929
580,US Sassuolo,Edoardo Goldaniga,23.0,Centre-Back,US Palermo,4.00m,in,Summer,4.000,Serie A,...,42,74,73,70,9,12,7,15,15,7.816261
581,SPAL 2013,Felipe,32.0,Centre-Back,Udinese Calcio,600Th.,in,Summer,0.600,Serie A,...,67,80,80,76,11,6,8,9,5,2.053159
582,Udinese Calcio,Felipe,32.0,Centre-Back,SPAL,600Th.,out,Summer,0.600,Serie A,...,67,80,80,76,11,6,8,9,5,2.053159


In [ ]:
new_column_order2017 = ['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Position', 'Overall', 'Potential', 'Value', 'Wage',
       'Nationality', 'preferred_foot', 'Work Rate', 'Body Type', 'Crossing',
       'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'fee_cleaned','fee_pred']
Final2017 = result_2017[new_column_order2017]

In [ ]:
Final2017

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_cleaned,fee_pred
0,FC Barcelona,Javier Mascherano,33.0,Defensive Midfield,HB CFFC,5.50m,out,Winter,5.500,Primera Division,...,84,87,85,6,10,8,5,5,5.500,9.932839
1,Atlético de Madrid,Diego Costa,29.0,Centre-Forward,Chelsea,60.00m,in,Winter,60.000,Primera Division,...,16,19,18,70,68,63,73,74,60.000,13.876005
2,Atlético de Madrid,Diego Costa,29.0,Centre-Forward,Chelsea,60.00m,in,Winter,60.000,Primera Division,...,28,39,34,11,13,12,8,11,60.000,50.927359
3,Atlético de Madrid,Diego Costa,29.0,Centre-Forward,Chelsea,60.00m,in,Winter,60.000,Primera Division,...,41,58,44,13,15,9,12,5,60.000,51.246503
4,Chelsea FC,Diego Costa,29.0,Centre-Forward,Atlético Madrid,60.00m,out,Winter,60.000,Premier League,...,16,19,18,70,68,63,73,74,60.000,13.876005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,AC Milan,Leonardo Bonucci,30.0,Centre-Back,Juventus,42.00m,in,Summer,42.000,Serie A,...,84,88,88,2,2,3,2,4,42.000,35.351929
580,US Sassuolo,Edoardo Goldaniga,23.0,Centre-Back,US Palermo,4.00m,in,Summer,4.000,Serie A,...,74,73,70,9,12,7,15,15,4.000,7.816261
581,SPAL 2013,Felipe,32.0,Centre-Back,Udinese Calcio,600Th.,in,Summer,0.600,Serie A,...,80,80,76,11,6,8,9,5,0.600,2.053159
582,Udinese Calcio,Felipe,32.0,Centre-Back,SPAL,600Th.,out,Summer,0.600,Serie A,...,80,80,76,11,6,8,9,5,0.600,2.053159


ทำนายราคานักเตะในปี2018

In [ ]:
result_2018 #พบว่าในปี2018 ชุดข้อมูลมีค่า Nan จึงจัดการก่อน(รู้มาจากการทำแล้วerror)

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,Athletic Bilbao,Ibai Gómez,29.0,Right Winger,Alavés,4.00m,in,Winter,4.00,Primera Division,...,65,67,45,58,55,14,8,13,7,11
1,Deportivo Alavés,Ibai Gómez,29.0,Right Winger,Athletic,4.00m,out,Winter,4.00,Primera Division,...,65,67,45,58,55,14,8,13,7,11
2,Athletic Bilbao,Kenan Kodro,25.0,Centre-Forward,FC Copenhagen,900Th.,in,Winter,0.90,Primera Division,...,60,55,15,41,37,7,14,15,6,12
3,1.FSV Mainz 05,Kenan Kodro,24.0,Centre-Forward,FC Copenhagen,1.60m,out,Summer,1.60,1 Bundesliga,...,60,55,15,41,37,7,14,15,6,12
4,Deportivo Alavés,Rubén Sobrino,26.0,Centre-Forward,Valencia,5.00m,out,Winter,5.00,Primera Division,...,66,61,19,31,17,13,8,11,7,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,Juventus FC,Mattia Caldara,24.0,Centre-Back,AC Milan,36.87m,out,Summer,36.87,Serie A,...,30,73,78,79,76,14,9,6,9,5
503,AC Milan,Mattia Caldara,24.0,Centre-Back,Juventus,36.87m,in,Summer,36.87,Serie A,...,30,73,78,79,76,14,9,6,9,5
504,SS Lazio,Francesco Acerbi,30.0,Centre-Back,Sassuolo,12.00m,in,Summer,12.00,Serie A,...,72,83,86,86,85,11,9,9,13,11
505,US Sassuolo,Francesco Acerbi,30.0,Centre-Back,Lazio,12.00m,out,Summer,12.00,Serie A,...,72,83,86,86,85,11,9,9,13,11


In [ ]:
result_2018 = result_2018.drop('Release Clause', axis=1)

In [ ]:
result_2018 = result_2018.dropna().reset_index(drop=True)
result_2018

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,Penalties,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,Athletic Bilbao,Ibai Gómez,29.0,Right Winger,Alavés,4.00m,in,Winter,4.00,Primera Division,...,65,67,45,58,55,14,8,13,7,11
1,Deportivo Alavés,Ibai Gómez,29.0,Right Winger,Athletic,4.00m,out,Winter,4.00,Primera Division,...,65,67,45,58,55,14,8,13,7,11
2,Athletic Bilbao,Kenan Kodro,25.0,Centre-Forward,FC Copenhagen,900Th.,in,Winter,0.90,Primera Division,...,60,55,15,41,37,7,14,15,6,12
3,1.FSV Mainz 05,Kenan Kodro,24.0,Centre-Forward,FC Copenhagen,1.60m,out,Summer,1.60,1 Bundesliga,...,60,55,15,41,37,7,14,15,6,12
4,Deportivo Alavés,Rubén Sobrino,26.0,Centre-Forward,Valencia,5.00m,out,Winter,5.00,Primera Division,...,66,61,19,31,17,13,8,11,7,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,Juventus FC,Mattia Caldara,24.0,Centre-Back,AC Milan,36.87m,out,Summer,36.87,Serie A,...,30,73,78,79,76,14,9,6,9,5
501,AC Milan,Mattia Caldara,24.0,Centre-Back,Juventus,36.87m,in,Summer,36.87,Serie A,...,30,73,78,79,76,14,9,6,9,5
502,SS Lazio,Francesco Acerbi,30.0,Centre-Back,Sassuolo,12.00m,in,Summer,12.00,Serie A,...,72,83,86,86,85,11,9,9,13,11
503,US Sassuolo,Francesco Acerbi,30.0,Centre-Back,Lazio,12.00m,out,Summer,12.00,Serie A,...,72,83,86,86,85,11,9,9,13,11


In [ ]:
batch2018= result_2018.drop(['club_name',
                              'club_involved_name',
                              'Nationality',
                              'transfer_movement',
                              'Id',
                              'Position',
                              'fee_cleaned',


                              'short_name',
                              'player_name',
                               'Name',
                              'position',
                              'fee',
                              'transfer_period',
                              'league_name',
                              'season',
                              'country',



                              'preferred_foot',
                              'Body Type',



                              'Composure',
                              'Work Rate',




                              'Team',
                              'year',


                              ], axis=1)
batch2018.describe()

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
count,505.000000,505.000000,505.000000,5.050000e+02,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,...,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000
mean,25.736634,75.122772,79.877228,1.022109e+07,42077.227723,58.300990,54.435644,59.778218,68.047525,51.859406,...,61.938614,55.756436,48.293069,53.128713,50.447525,15.891089,15.510891,15.035644,15.489109,15.687129
std,3.615984,5.397749,4.493467,1.079407e+07,49695.541066,19.288034,21.001519,17.774110,13.523558,19.614692,...,14.618637,16.207548,23.994238,24.270247,24.590974,18.019776,17.740311,15.802436,17.848246,18.730999
min,18.000000,53.000000,67.000000,1.200000e+05,2000.000000,9.000000,6.000000,10.000000,11.000000,6.000000,...,13.000000,12.000000,5.000000,10.000000,10.000000,2.000000,2.000000,3.000000,2.000000,3.000000
25%,23.000000,72.000000,77.000000,3.600000e+06,15000.000000,47.000000,38.000000,53.000000,64.000000,37.000000,...,53.000000,46.000000,22.000000,30.000000,25.000000,9.000000,8.000000,8.000000,8.000000,8.000000
50%,26.000000,75.000000,80.000000,7.500000e+06,30000.000000,64.000000,60.000000,64.000000,71.000000,55.000000,...,65.000000,58.000000,57.000000,63.000000,60.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,28.000000,78.000000,83.000000,1.300000e+07,50000.000000,73.000000,71.000000,72.000000,77.000000,69.000000,...,73.000000,68.000000,70.000000,74.000000,73.000000,14.000000,14.000000,14.000000,14.000000,14.000000
max,36.000000,94.000000,94.000000,9.550000e+07,575000.000000,86.000000,94.000000,91.000000,91.000000,88.000000,...,91.000000,91.000000,86.000000,89.000000,88.000000,85.000000,91.000000,79.000000,86.000000,90.000000


In [ ]:
batch2018 = batch2018.dropna()

In [ ]:
batch2018

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,29.0,76,76,8000000.0,30000,77,73,47,73,75,...,75,65,45,58,55,14,8,13,7,11
1,29.0,76,76,8000000.0,30000,77,73,47,73,75,...,75,65,45,58,55,14,8,13,7,11
2,25.0,71,78,3400000.0,20000,64,73,73,65,66,...,65,60,15,41,37,7,14,15,6,12
3,24.0,71,78,3400000.0,20000,64,73,73,65,66,...,65,60,15,41,37,7,14,15,6,12
4,26.0,70,75,2400000.0,15000,55,68,56,64,59,...,54,66,19,31,17,13,8,11,7,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,24.0,76,85,10500000.0,20000,31,34,74,55,40,...,22,30,78,79,76,14,9,6,9,5
501,24.0,76,85,10500000.0,20000,31,34,74,55,40,...,22,30,78,79,76,14,9,6,9,5
502,30.0,82,82,17000000.0,70000,25,37,77,70,39,...,59,72,86,86,85,11,9,9,13,11
503,30.0,82,82,17000000.0,70000,25,37,77,70,39,...,59,72,86,86,85,11,9,9,13,11


In [ ]:
test2018 = scale.fit_transform(batch2018)

test2018.shape

(505, 38)

In [ ]:
batch_pca2018 = pca.fit_transform(test2018)
batch_pca2018

array([[-3.01414678, -1.48680353, -0.81048639, ..., -0.02213323,
         0.12172786,  0.03854013],
       [-3.01414678, -1.48680353, -0.81048639, ..., -0.02213323,
         0.12172786,  0.03854013],
       [ 0.17133236, -0.55366502, -1.33198301, ..., -0.46072803,
         0.17952393, -0.1192312 ],
       ...,
       [ 0.23994021,  3.77144621,  3.19254631, ...,  0.33979969,
        -0.12567569,  0.10532368],
       [ 0.23994021,  3.77144621,  3.19254631, ...,  0.33979969,
        -0.12567569,  0.10532368],
       [ 4.01752896,  4.32443407, -0.89841232, ..., -0.1299111 ,
         0.19406181, -0.23306322]])

In [ ]:
y_pred2018 = gb_model.predict(test2018)
y_pred2018

array([  4.44614954,   4.44614954,   4.63104538,   4.63104538,
         6.50073256,   6.50073256,   9.56994792,  22.13928323,
         9.48409929,  19.62767194,  15.10780778,  15.10780778,
        64.22571898,  64.22571898,   6.9038688 ,   9.02421147,
         9.02421147,   4.5959032 ,   1.29734865,  30.62052401,
        30.62052401,  21.29631483,  21.29631483,   3.18953832,
        22.19039577,   7.35660558,  28.99569223,   7.35660558,
        28.99569223,   3.18953832,  22.19039577,  22.67740412,
        22.67740412,   9.25157954,  18.74449128,   9.25157954,
        18.74449128,   5.44813821,   5.44813821,   7.49434669,
         7.49434669,   6.52887529,   4.15467712,   1.48191939,
         1.48191939,   3.88622127,   7.370179  ,   3.88622127,
         9.25349517,   5.07174658,   5.07174658,  11.98524349,
         4.04760614,  10.05128278,   5.00660994,   5.00660994,
         5.09084556,  11.42029962,  16.26377346,  16.26377346,
         8.81168727,   8.81168727,  16.51295126,  16.51

In [ ]:
fee_pred2018 = pd.DataFrame(y_pred2018)
fee_pred2018 = fee_pred2018.rename(columns={0: 'fee_pred'})

In [ ]:
fee_pred2018

,fee_pred
0,4.446150
1,4.446150
2,4.631045
3,4.631045
4,6.500733
...,...
500,23.902433
501,23.902433
502,8.454159
503,8.454159


In [ ]:
fee_pred2018 = fee_pred2018.dropna()
fee_pred2018

,fee_pred
0,4.446150
1,4.446150
2,4.631045
3,4.631045
4,6.500733
...,...
500,23.902433
501,23.902433
502,8.454159
503,8.454159


In [ ]:
nan_count_column = fee_pred2018.isna().sum()  # นับค่า NaN ในแต่ละคอลัมน์
nan_count_row = fee_pred2018.isna().sum(axis=1)  # นับค่า NaN ในแต่ละแถว

print("Count of NaN in each column:")
print(nan_count_column)

print("\nCount of NaN in each row:")
print(nan_count_row)

Count of NaN in each column:
fee_pred    0
dtype: int64

Count of NaN in each row:
0      0
1      0
2      0
3      0
4      0
      ..
500    0
501    0
502    0
503    0
504    0
Length: 505, dtype: int64


In [ ]:
batch2018 = batch2018.copy()
batch2018['fee_pred'] = fee_pred2018


In [ ]:
result_2018['fee_pred'] = fee_pred2018
result_2018.columns

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Position', 'Overall', 'Potential', 'Value', 'Wage', 'Team',
       'Nationality', 'preferred_foot', 'Work Rate', 'Body Type', 'Crossing',
       'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'fee_pred'],
      dtype='object')

In [ ]:
new_column_order2018 = ['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Position', 'Overall', 'Potential', 'Value', 'Wage', 'Team',
       'Nationality', 'preferred_foot', 'Work Rate', 'Body Type',
        'Crossing', 'Finishing', 'Heading accuracy',
       'Short passing', 'Volleys', 'Dribbling', 'Curve', 'Fk accuracy',
       'Long passing', 'Ball control', 'Acceleration', 'Sprint speed',
       'Agility', 'Reactions', 'Balance', 'Shot power', 'Jumping', 'Stamina',
       'Strength', 'Long shots', 'Aggression', 'Interceptions', 'Positioning',
       'Vision', 'Penalties', 'Composure', 'Defensive awareness',
       'Standing tackle', 'Sliding tackle', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'fee_cleaned','fee_pred']
Final2018 = result_2018[new_column_order2018]

In [ ]:
Final2018

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,league_name,year,...,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_cleaned,fee_pred
0,Athletic Bilbao,Ibai Gómez,29.0,Right Winger,Alavés,4.00m,in,Winter,Primera Division,2018,...,45,58,55,14,8,13,7,11,4.00,4.446150
1,Deportivo Alavés,Ibai Gómez,29.0,Right Winger,Athletic,4.00m,out,Winter,Primera Division,2018,...,45,58,55,14,8,13,7,11,4.00,4.446150
2,Athletic Bilbao,Kenan Kodro,25.0,Centre-Forward,FC Copenhagen,900Th.,in,Winter,Primera Division,2018,...,15,41,37,7,14,15,6,12,0.90,4.631045
3,1.FSV Mainz 05,Kenan Kodro,24.0,Centre-Forward,FC Copenhagen,1.60m,out,Summer,1 Bundesliga,2018,...,15,41,37,7,14,15,6,12,1.60,4.631045
4,Deportivo Alavés,Rubén Sobrino,26.0,Centre-Forward,Valencia,5.00m,out,Winter,Primera Division,2018,...,19,31,17,13,8,11,7,15,5.00,6.500733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,Juventus FC,Mattia Caldara,24.0,Centre-Back,AC Milan,36.87m,out,Summer,Serie A,2018,...,78,79,76,14,9,6,9,5,36.87,23.902433
501,AC Milan,Mattia Caldara,24.0,Centre-Back,Juventus,36.87m,in,Summer,Serie A,2018,...,78,79,76,14,9,6,9,5,36.87,23.902433
502,SS Lazio,Francesco Acerbi,30.0,Centre-Back,Sassuolo,12.00m,in,Summer,Serie A,2018,...,86,86,85,11,9,9,13,11,12.00,8.454159
503,US Sassuolo,Francesco Acerbi,30.0,Centre-Back,Lazio,12.00m,out,Summer,Serie A,2018,...,86,86,85,11,9,9,13,11,12.00,8.454159


ทำนายราคานักเตะในปี2019

In [ ]:
batch2019 = result_2019.drop(['club_name',
                              'club_involved_name',
                              'Nationality',
                              'transfer_movement',
                              'Id',
                              'Position',
                              'fee_cleaned',
                              'Release Clause',

                              'short_name',
                              'player_name',
                               'Name',
                              'position',
                              'fee',
                              'transfer_period',
                              'league_name',
                              'season',
                              'country',



                              'preferred_foot',
                              'Body Type',



                              'Composure',
                              'Work Rate',




                              'Team',
                              'year',


                              ], axis=1)
batch2019.describe()

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
count,590.000000,590.000000,590.000000,5.900000e+02,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,...,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000,590.000000
mean,25.089831,74.811864,80.262712,1.003518e+07,37132.203390,59.457627,55.498305,59.747458,68.498305,53.488136,...,62.328814,55.083051,53.467797,53.313559,50.113559,14.596610,14.325424,14.367797,14.691525,14.728814
std,3.583645,5.490247,4.497930,1.130942e+07,39073.777418,18.013032,20.230316,16.579559,13.420931,18.309238,...,14.125812,15.295778,21.412628,24.281002,24.516631,16.816497,15.892226,15.126248,16.198255,17.031211
min,18.000000,52.000000,62.000000,9.000000e+04,2000.000000,7.000000,5.000000,10.000000,14.000000,6.000000,...,15.000000,12.000000,7.000000,10.000000,10.000000,3.000000,3.000000,3.000000,3.000000,3.000000
25%,22.000000,72.000000,77.000000,3.200000e+06,15000.000000,51.000000,41.000000,52.000000,64.000000,43.000000,...,55.000000,46.000000,31.250000,28.000000,25.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,25.000000,75.000000,80.000000,8.000000e+06,25000.000000,64.000000,61.000000,62.500000,72.000000,59.000000,...,65.000000,58.000000,61.000000,63.000000,58.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,28.000000,78.000000,83.000000,1.250000e+07,40000.000000,72.000000,71.000000,72.000000,77.000000,67.000000,...,73.000000,66.000000,72.000000,75.000000,72.750000,13.000000,13.000000,14.000000,14.000000,14.000000
max,36.000000,91.000000,91.000000,9.300000e+07,325000.000000,91.000000,92.000000,90.000000,91.000000,90.000000,...,91.000000,90.000000,89.000000,92.000000,90.000000,90.000000,84.000000,79.000000,83.000000,90.000000


In [ ]:
batch2019

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,24.0,75,81,8000000.0,15000,38,36,72,62,36,...,44,32,77,75,75,11,7,13,14,6
1,24.0,75,81,8000000.0,15000,38,36,72,62,36,...,44,32,77,75,75,11,7,13,14,6
2,27.0,74,77,6000000.0,20000,48,52,70,67,72,...,54,48,72,72,76,15,11,6,14,12
3,27.0,74,77,6000000.0,20000,48,52,70,67,72,...,54,48,72,72,76,15,11,6,14,12
4,28.0,72,75,3200000.0,15000,60,23,79,69,27,...,45,49,55,75,78,10,16,10,15,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,34.0,76,76,3100000.0,15000,35,29,76,58,34,...,28,45,80,78,77,5,5,5,5,5
586,36.0,75,75,1600000.0,10000,25,36,78,45,35,...,25,33,76,78,74,8,7,10,10,14
587,28.0,85,87,37500000.0,100000,27,10,80,60,16,...,22,26,86,87,89,8,6,8,15,12
588,28.0,85,87,37500000.0,100000,27,10,80,60,16,...,22,26,86,87,89,8,6,8,15,12


In [ ]:
test2019 = scale.fit_transform(batch2019)

test2019.shape

(590, 38)

In [ ]:
batch_pca2019 = pca.fit_transform(test2019)
batch_pca2019

array([[ 2.25983777,  3.94492882, -0.4980083 , ...,  0.30646972,
         0.07044454,  0.35258466],
       [ 2.25983777,  3.94492882, -0.4980083 , ...,  0.30646972,
         0.07044454,  0.35258466],
       [ 0.46863776,  2.77930929, -0.35710144, ...,  0.66253667,
        -0.07839675, -0.08601202],
       ...,
       [ 2.24008269,  5.53579129,  3.04056929, ..., -0.52765802,
         0.5175845 ,  0.10766273],
       [ 2.24008269,  5.53579129,  3.04056929, ..., -0.52765802,
         0.5175845 ,  0.10766273],
       [ 6.6051596 ,  2.08816604, -5.39540865, ...,  0.32885363,
        -0.20089005, -0.14848566]])

In [ ]:
y_pred2019 = gb_model.predict(test2019)
y_pred2019

array([  7.57191626,   7.57191626,   3.68120163,   3.68120163,
         5.24796192,   5.24796192,  26.93027111,  26.93027111,
         9.04701434,   9.04701434,   1.12989155,   4.74121374,
        18.26903195,  18.26903195,  18.26903195,  18.26903195,
        15.24681558,  15.24681558, 100.65898308, 100.65898308,
        18.61778314,  18.61778314,  10.17884006,  10.17884006,
        50.83927449,  14.01797705,  14.01797705,  13.4893327 ,
        22.42662947,  13.4893327 ,  22.42662947,  20.88249232,
         4.35176221,  20.88249232,   4.35176221,   4.46186805,
         4.46186805,  28.65284532,  28.65284532,  23.64071984,
        23.64071984,   1.91386105,   4.20292535,  58.37105685,
        58.37105685,  47.76997725,  47.76997725,  40.82255237,
        40.82255237,  91.3457774 ,  91.3457774 ,  20.17052608,
        20.17052608,  19.45087515,  19.45087515,  17.77874106,
        17.77874106,   1.2784937 ,   3.05394602,   4.73564079,
         4.73564079,   7.2495855 ,   7.2495855 ,  18.25

In [ ]:
fee_pred2019 = pd.DataFrame(y_pred2019)
fee_pred2019 = fee_pred2019.rename(columns={0: 'fee_pred'})

In [ ]:
fee_pred2019

,fee_pred
0,7.571916
1,7.571916
2,3.681202
3,3.681202
4,5.247962
...,...
585,2.532573
586,3.657057
587,40.076262
588,40.076262


In [ ]:
batch2019['fee_pred'] = fee_pred2019
batch2019.columns
batch2019

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_pred
0,24.0,75,81,8000000.0,15000,38,36,72,62,36,...,32,77,75,75,11,7,13,14,6,7.571916
1,24.0,75,81,8000000.0,15000,38,36,72,62,36,...,32,77,75,75,11,7,13,14,6,7.571916
2,27.0,74,77,6000000.0,20000,48,52,70,67,72,...,48,72,72,76,15,11,6,14,12,3.681202
3,27.0,74,77,6000000.0,20000,48,52,70,67,72,...,48,72,72,76,15,11,6,14,12,3.681202
4,28.0,72,75,3200000.0,15000,60,23,79,69,27,...,49,55,75,78,10,16,10,15,16,5.247962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,34.0,76,76,3100000.0,15000,35,29,76,58,34,...,45,80,78,77,5,5,5,5,5,2.532573
586,36.0,75,75,1600000.0,10000,25,36,78,45,35,...,33,76,78,74,8,7,10,10,14,3.657057
587,28.0,85,87,37500000.0,100000,27,10,80,60,16,...,26,86,87,89,8,6,8,15,12,40.076262
588,28.0,85,87,37500000.0,100000,27,10,80,60,16,...,26,86,87,89,8,6,8,15,12,40.076262


In [ ]:
result_2019['fee_pred'] = fee_pred2019
result_2019.columns

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Nationality', 'Position', 'Overall', 'Potential', 'Value',
       'Wage', 'preferred_foot', 'Work Rate', 'Body Type', 'Release Clause',
       'Crossing', 'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'fee_pred'],
      dtype='object')

In [ ]:
result_2019

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,Composure,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_pred
0,Getafe CF,Erick Cabaco,24.0,Centre-Back,Levante,8.00m,in,Winter,8.0,Primera Division,...,66,77,75,75,11,7,13,14,6,7.571916
1,Levante UD,Erick Cabaco,24.0,Centre-Back,Getafe,8.00m,out,Winter,8.0,Primera Division,...,66,77,75,75,11,7,13,14,6,7.571916
2,Getafe CF,Chema Rodríguez,27.0,Centre-Back,Nottm Forest,1.50m,in,Winter,1.5,Primera Division,...,65,72,72,76,15,11,6,14,12,3.681202
3,Levante UD,Chema Rodríguez,27.0,Centre-Back,Nottm Forest,500Th.,out,Summer,0.5,Primera Division,...,65,72,72,76,15,11,6,14,12,3.681202
4,Getafe CF,Leandro Cabrera,28.0,Centre-Back,Espanyol,9.00m,out,Winter,9.0,Primera Division,...,56,55,75,78,10,16,10,15,16,5.247962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,US Lecce,Luca Rossettini,34.0,Centre-Back,Genoa,500Th.,in,Summer,0.5,Serie A,...,74,80,78,77,5,5,5,5,5,2.532573
586,Genoa CFC,Nicolás Spolli,36.0,Centre-Back,Crotone,400Th.,out,Summer,0.4,Serie A,...,73,76,78,74,8,7,10,10,14,3.657057
587,SSC Napoli,Konstantinos Manolas,28.0,Centre-Back,AS Roma,36.00m,in,Summer,36.0,Serie A,...,79,86,87,89,8,6,8,15,12,40.076262
588,AS Roma,Konstantinos Manolas,28.0,Centre-Back,SSC Napoli,36.00m,out,Summer,36.0,Serie A,...,79,86,87,89,8,6,8,15,12,40.076262


In [ ]:
new_column_order2019 = ['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Nationality', 'Position', 'Overall', 'Potential', 'Value',
       'Wage', 'preferred_foot', 'Work Rate', 'Body Type', 'Release Clause',
       'Crossing', 'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'fee_cleaned','fee_pred']
Final2019 = result_2019[new_column_order2019]

In [ ]:
Final2019

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,league_name,year,...,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_cleaned,fee_pred
0,Getafe CF,Erick Cabaco,24.0,Centre-Back,Levante,8.00m,in,Winter,Primera Division,2019,...,77,75,75,11,7,13,14,6,8.0,7.571916
1,Levante UD,Erick Cabaco,24.0,Centre-Back,Getafe,8.00m,out,Winter,Primera Division,2019,...,77,75,75,11,7,13,14,6,8.0,7.571916
2,Getafe CF,Chema Rodríguez,27.0,Centre-Back,Nottm Forest,1.50m,in,Winter,Primera Division,2019,...,72,72,76,15,11,6,14,12,1.5,3.681202
3,Levante UD,Chema Rodríguez,27.0,Centre-Back,Nottm Forest,500Th.,out,Summer,Primera Division,2019,...,72,72,76,15,11,6,14,12,0.5,3.681202
4,Getafe CF,Leandro Cabrera,28.0,Centre-Back,Espanyol,9.00m,out,Winter,Primera Division,2019,...,55,75,78,10,16,10,15,16,9.0,5.247962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,US Lecce,Luca Rossettini,34.0,Centre-Back,Genoa,500Th.,in,Summer,Serie A,2019,...,80,78,77,5,5,5,5,5,0.5,2.532573
586,Genoa CFC,Nicolás Spolli,36.0,Centre-Back,Crotone,400Th.,out,Summer,Serie A,2019,...,76,78,74,8,7,10,10,14,0.4,3.657057
587,SSC Napoli,Konstantinos Manolas,28.0,Centre-Back,AS Roma,36.00m,in,Summer,Serie A,2019,...,86,87,89,8,6,8,15,12,36.0,40.076262
588,AS Roma,Konstantinos Manolas,28.0,Centre-Back,SSC Napoli,36.00m,out,Summer,Serie A,2019,...,86,87,89,8,6,8,15,12,36.0,40.076262


ทำนายราคานักเตะในปี2020

In [ ]:
batch2020 = result_2020.drop(['club_name',
                              'club_involved_name',
                              'Nationality',
                              'transfer_movement',
                              'Id',
                              'Position',
                              'fee_cleaned',
                              'Release Clause',

                              'short_name',
                              'player_name',
                               'Name',
                              'position',
                              'fee',
                              'transfer_period',
                              'league_name',
                              'season',
                              'country',



                              'preferred_foot',
                              'Body Type',



                              'Composure',
                              'Work Rate',




                              'Team',
                              'year',


                              ], axis=1)
batch2020.describe()

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
count,368.000000,368.000000,368.000000,3.680000e+02,368.000000,368.000000,368.000000,368.000000,368.000000,368.000000,...,368.000000,368.000000,368.000000,368.000000,368.000000,368.000000,368.000000,368.000000,368.00000,368.000000
mean,25.570652,74.595109,79.983696,1.012698e+07,39248.777174,57.230978,55.095109,60.277174,68.576087,51.861413,...,62.032609,54.747283,54.423913,54.567935,52.323370,15.353261,15.285326,14.711957,15.25000,15.467391
std,3.580968,5.743061,4.597684,1.050820e+07,44055.192871,18.694837,20.192781,17.419940,13.397697,18.935135,...,14.518766,15.892477,21.501275,24.453721,24.243346,17.102040,16.443035,15.826672,16.47796,17.619781
min,18.000000,59.000000,64.000000,2.300000e+05,550.000000,9.000000,5.000000,10.000000,18.000000,6.000000,...,20.000000,11.000000,7.000000,10.000000,9.000000,5.000000,4.000000,3.000000,5.00000,5.000000
25%,23.000000,71.000000,77.000000,3.400000e+06,15000.000000,43.000000,39.750000,53.000000,64.750000,38.000000,...,54.000000,45.000000,36.000000,31.000000,28.000000,9.000000,8.000000,8.000000,8.00000,8.000000
50%,25.000000,75.000000,80.000000,8.000000e+06,28000.000000,64.000000,59.500000,64.000000,72.000000,55.000000,...,65.000000,56.000000,63.000000,66.000000,64.000000,11.000000,11.000000,11.000000,11.00000,11.000000
75%,28.000000,78.000000,83.000000,1.200000e+07,47000.000000,72.000000,71.000000,72.250000,77.000000,67.000000,...,73.000000,67.000000,72.000000,75.250000,73.000000,14.000000,14.000000,14.000000,14.00000,13.000000
max,35.000000,89.000000,92.000000,6.100000e+07,350000.000000,90.000000,91.000000,90.000000,90.000000,90.000000,...,86.000000,91.000000,88.000000,88.000000,86.000000,83.000000,77.000000,77.000000,77.00000,86.000000


In [ ]:
batch2020.columns

Index(['age', 'Overall', 'Potential', 'Value', 'Wage', 'Crossing', 'Finishing',
       'Heading accuracy', 'Short passing', 'Volleys', 'Dribbling', 'Curve',
       'Fk accuracy', 'Long passing', 'Ball control', 'Acceleration',
       'Sprint speed', 'Agility', 'Reactions', 'Balance', 'Shot power',
       'Jumping', 'Stamina', 'Strength', 'Long shots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Defensive awareness', 'Standing tackle', 'Sliding tackle', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes'],
      dtype='object')

In [ ]:
batch2020

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,27.0,81,84,21500000.0,45000,65,69,75,83,62,...,79,45,77,80,81,15,10,10,8,10
1,27.0,81,84,21500000.0,45000,65,69,75,83,62,...,79,45,77,80,81,15,10,10,8,10
2,24.0,75,84,10500000.0,15000,40,80,73,69,70,...,67,70,21,21,22,12,13,8,9,12
3,24.0,75,84,10500000.0,15000,40,80,73,69,70,...,67,70,21,21,22,12,13,8,9,12
4,32.0,80,80,12000000.0,68000,69,69,72,79,69,...,71,54,76,79,77,12,11,11,14,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,28.0,80,83,16000000.0,43000,86,59,64,78,67,...,74,83,80,78,78,14,13,14,14,11
364,30.0,77,77,7500000.0,53000,40,25,74,54,33,...,30,35,79,81,79,13,11,13,11,6
365,30.0,77,77,7500000.0,53000,40,25,74,54,33,...,30,35,79,81,79,13,11,13,11,6
366,25.0,77,80,10000000.0,25000,68,35,73,75,41,...,65,47,77,77,73,16,10,8,14,10


In [ ]:
test2020 = scale.fit_transform(batch2020)

test2020.shape

(368, 38)

In [ ]:
batch_pca2020 = pca.fit_transform(test2020)
batch_pca2020

array([[-3.81221803,  1.86597325,  2.70755812, ..., -0.03351888,
        -0.24621241, -0.06722051],
       [-3.81221803,  1.86597325,  2.70755812, ..., -0.03351888,
        -0.24621241, -0.06722051],
       [-0.45924131, -2.22613864, -0.44292291, ...,  0.10084909,
         0.07313417, -0.07531556],
       ...,
       [ 2.42119828,  4.0266848 ,  0.04773669, ..., -0.09612346,
         0.00544907,  0.09335437],
       [-1.61910854,  2.13680551,  1.05337653, ...,  0.10424041,
        -0.36951636, -0.66390878],
       [ 3.63231271,  3.53998511, -1.57555928, ..., -0.44047051,
         0.13855313, -0.20753619]])

In [ ]:
y_pred2020 = gb_model.predict(test2020)
y_pred2020

array([69.32674139, 69.32674139, 16.56950862, 16.56950862,  5.60294178,
       19.82911571, 19.82911571, 28.77026327, 28.77026327, 11.23212034,
        7.52993658, 11.23212034,  7.52993658, 59.72092042, 59.72092042,
       39.17021251, 39.17021251, 13.37106683, 13.37106683, 18.93110352,
       18.93110352, 30.24623489, 30.24623489,  9.90924488,  8.24873647,
        8.24873647, 12.65087986, 12.65087986,  6.78690575,  0.87927023,
        0.87927023,  4.84468069,  4.84468069,  7.08321609, 12.69576041,
       12.69576041,  0.84656549,  0.84656549, 17.7130068 , 17.7130068 ,
       34.43357147, 34.43357147, 12.67549564, 12.67549564,  1.13523433,
        1.13523433,  5.27284018,  9.73197007,  9.73197007,  6.43528541,
        2.53039101,  9.0963041 ,  9.0963041 , 38.94543043, 38.94543043,
        2.77900854,  4.88088867,  4.88088867, 17.19527622, 17.19527622,
       10.05304015,  3.08985819, 17.94794998, 28.70866918, 28.70866918,
       11.63472656, 11.63472656,  5.42947471,  5.42947471,  6.67

In [ ]:
fee_pred2020 = pd.DataFrame(y_pred2020)
fee_pred2020 = fee_pred2020.rename(columns={0: 'fee_pred'})

In [ ]:
fee_pred2020

,fee_pred
0,69.326741
1,69.326741
2,16.569509
3,16.569509
4,5.602942
...,...
363,13.198152
364,2.896908
365,2.896908
366,9.859009


In [ ]:
batch2020['fee_pred'] = fee_pred2020
batch2020.columns
batch2020

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_pred
0,27.0,81,84,21500000.0,45000,65,69,75,83,62,...,45,77,80,81,15,10,10,8,10,69.326741
1,27.0,81,84,21500000.0,45000,65,69,75,83,62,...,45,77,80,81,15,10,10,8,10,69.326741
2,24.0,75,84,10500000.0,15000,40,80,73,69,70,...,70,21,21,22,12,13,8,9,12,16.569509
3,24.0,75,84,10500000.0,15000,40,80,73,69,70,...,70,21,21,22,12,13,8,9,12,16.569509
4,32.0,80,80,12000000.0,68000,69,69,72,79,69,...,54,76,79,77,12,11,11,14,12,5.602942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,28.0,80,83,16000000.0,43000,86,59,64,78,67,...,83,80,78,78,14,13,14,14,11,13.198152
364,30.0,77,77,7500000.0,53000,40,25,74,54,33,...,35,79,81,79,13,11,13,11,6,2.896908
365,30.0,77,77,7500000.0,53000,40,25,74,54,33,...,35,79,81,79,13,11,13,11,6,2.896908
366,25.0,77,80,10000000.0,25000,68,35,73,75,41,...,47,77,77,73,16,10,8,14,10,9.859009


In [ ]:
result_2020['fee_pred'] = fee_pred2020
result_2020.columns

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Position', 'Overall', 'Potential', 'Value', 'Wage',
       'Nationality', 'preferred_foot', 'Work Rate', 'Body Type',
       'Release Clause', 'Crossing', 'Finishing', 'Heading accuracy',
       'Short passing', 'Volleys', 'Dribbling', 'Curve', 'Fk accuracy',
       'Long passing', 'Ball control', 'Acceleration', 'Sprint speed',
       'Agility', 'Reactions', 'Balance', 'Shot power', 'Jumping', 'Stamina',
       'Strength', 'Long shots', 'Aggression', 'Interceptions', 'Positioning',
       'Vision', 'Penalties', 'Composure', 'Defensive awareness',
       'Standing tackle', 'Sliding tackle', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'fee_pred'],
      dtype='object')

In [ ]:
new_column_order2020 = ['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Position', 'Overall', 'Potential', 'Value', 'Wage',
       'Nationality', 'preferred_foot', 'Work Rate', 'Body Type',
       'Release Clause', 'Crossing', 'Finishing', 'Heading accuracy',
       'Short passing', 'Volleys', 'Dribbling', 'Curve', 'Fk accuracy',
       'Long passing', 'Ball control', 'Acceleration', 'Sprint speed',
       'Agility', 'Reactions', 'Balance', 'Shot power', 'Jumping', 'Stamina',
       'Strength', 'Long shots', 'Aggression', 'Interceptions', 'Positioning',
       'Vision', 'Penalties', 'Composure', 'Defensive awareness',
       'Standing tackle', 'Sliding tackle', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'fee_cleaned','fee_pred']
Final2020 = result_2020[new_column_order2020]

In [ ]:
Final2020

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,league_name,year,...,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_cleaned,fee_pred
0,Atlético de Madrid,Geoffrey Kondogbia,27.0,Defensive Midfield,Valencia,15.00m,in,Winter,Primera Division,2020,...,77,80,81,15,10,10,8,10,15.00,69.326741
1,Valencia CF,Geoffrey Kondogbia,27.0,Defensive Midfield,Atlético Madrid,15.00m,out,Winter,Primera Division,2020,...,77,80,81,15,10,10,8,10,15.00,69.326741
2,Sevilla FC,Carlos Fernández,24.0,Centre-Forward,Real Sociedad,10.00m,out,Winter,Primera Division,2020,...,21,21,22,12,13,8,9,12,10.00,16.569509
3,Real Sociedad,Carlos Fernández,24.0,Centre-Forward,Sevilla FC,10.00m,in,Winter,Primera Division,2020,...,21,21,22,12,13,8,9,12,10.00,16.569509
4,Villarreal CF,Étienne Capoue,32.0,Defensive Midfield,Watford,2.00m,in,Winter,Primera Division,2020,...,76,79,77,12,11,11,14,12,2.00,5.602942
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,Torino FC,Ricardo Rodríguez,28.0,Centre-Back,AC Milan,1.95m,in,Summer,Serie A,2020,...,80,78,78,14,13,14,14,11,1.95,13.198152
364,SSC Napoli,Lorenzo Tonelli,30.0,Centre-Back,Sampdoria,2.00m,out,Summer,Serie A,2020,...,79,81,79,13,11,13,11,6,2.00,2.896908
365,UC Sampdoria,Lorenzo Tonelli,30.0,Centre-Back,SSC Napoli,2.00m,in,Summer,Serie A,2020,...,79,81,79,13,11,13,11,6,2.00,2.896908
366,US Sassuolo,Kaan Ayhan,25.0,Centre-Back,F. Düsseldorf,2.50m,in,Summer,Serie A,2020,...,77,77,73,16,10,8,14,10,2.50,9.859009


ทำนายราคานักเตะในปี2021

In [ ]:
batch2021 = result_2021.drop(['club_name',
                              'club_involved_name',
                              'Nationality',
                              'transfer_movement',

                              'Position',
                              'fee_cleaned',

                              'short_name',
                              'player_name',
                               'Name',
                              'position',
                              'fee',
                              'transfer_period',
                              'league_name',
                              'season',
                              'country',
                              'Release Clause',



                              'preferred_foot',
                              'Body Type',



                              'Composure',
                              'Work Rate',




                              'Team',
                              'year',


                              ], axis=1)
batch2021.describe()

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
count,385.000000,385.000000,385.000000,3.850000e+02,385.000000,385.000000,385.000000,385.000000,385.000000,385.000000,...,385.000000,385.000000,385.000000,385.000000,385.000000,385.000000,385.000000,385.000000,385.000000,385.000000
mean,25.394805,74.192208,79.709091,1.290695e+07,36675.324675,57.610390,52.802597,59.838961,67.610390,49.815584,...,61.096104,52.633766,53.371429,54.464935,51.449351,15.501299,15.415584,14.937662,15.457143,16.220779
std,3.640051,5.814802,4.669666,1.610756e+07,35246.524488,19.176915,21.668610,18.110052,13.540791,20.222583,...,14.732108,16.938566,21.233320,23.170663,23.535062,18.038894,17.061432,16.418399,17.523759,18.429499
min,18.000000,54.000000,64.000000,2.000000e+05,500.000000,10.000000,6.000000,10.000000,15.000000,6.000000,...,13.000000,12.000000,6.000000,8.000000,8.000000,5.000000,4.000000,5.000000,4.000000,5.000000
25%,23.000000,71.000000,77.000000,3.200000e+06,15000.000000,46.000000,34.000000,54.000000,64.000000,33.000000,...,54.000000,40.000000,32.000000,32.000000,28.000000,8.000000,8.000000,8.000000,8.000000,9.000000
50%,25.000000,75.000000,79.000000,7.000000e+06,26000.000000,63.000000,59.000000,63.000000,71.000000,53.000000,...,64.000000,55.000000,59.000000,64.000000,60.000000,11.000000,11.000000,11.000000,11.000000,11.000000
75%,28.000000,78.000000,83.000000,1.550000e+07,48000.000000,72.000000,72.000000,72.000000,76.000000,67.000000,...,73.000000,65.000000,72.000000,75.000000,73.000000,13.000000,14.000000,13.000000,14.000000,14.000000
max,36.000000,92.000000,93.000000,1.240000e+08,220000.000000,90.000000,95.000000,90.000000,88.000000,90.000000,...,87.000000,92.000000,89.000000,89.000000,87.000000,86.000000,82.000000,93.000000,86.000000,88.000000


In [ ]:
batch2021

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,32.0,80,80,15500000.0,38000,79,75,69,80,71,...,78,64,79,78,77,11,9,10,13,13
1,32.0,80,80,15500000.0,38000,79,75,69,80,71,...,78,64,79,78,77,11,9,10,13,13
2,31.0,83,83,32000000.0,71000,86,48,74,80,58,...,80,66,77,83,81,11,14,8,11,10
3,31.0,83,83,32000000.0,71000,86,48,74,80,58,...,80,66,77,83,81,11,14,8,11,10
4,21.0,81,89,56500000.0,95000,77,77,70,80,72,...,86,66,35,24,28,12,9,11,15,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,28.0,77,79,11000000.0,26000,46,30,77,72,32,...,55,52,80,78,81,11,16,11,15,13
381,24.0,59,67,475000.0,6000,34,27,55,39,31,...,31,40,61,66,71,7,7,5,10,15
382,24.0,69,75,2000000.0,7000,28,22,65,62,23,...,38,31,71,72,70,9,14,10,12,13
383,24.0,69,75,2000000.0,7000,28,22,65,62,23,...,38,31,71,72,70,9,14,10,12,13


In [ ]:
test2021 = scale.fit_transform(batch2021)

test2021.shape

(385, 38)

In [ ]:
batch_pca2021 = pca.fit_transform(test2021)
batch_pca2021

array([[-4.52172248, -0.18892796,  1.78330661, ...,  0.19013178,
        -0.32092265,  0.0289072 ],
       [-4.52172248, -0.18892796,  1.78330661, ...,  0.19013178,
        -0.32092265,  0.0289072 ],
       [-4.45717041, -0.37740816,  2.72704181, ..., -0.08647772,
         0.1769749 ,  0.18654767],
       ...,
       [ 2.68391909, -3.37780382, -1.11608751, ..., -0.38707953,
        -0.13434237, -0.08709194],
       [ 2.68391909, -3.37780382, -1.11608751, ..., -0.38707953,
        -0.13434237, -0.08709194],
       [ 0.51728782, -2.68655106, -0.05763946, ...,  0.49325848,
         0.29759301, -0.09097456]])

In [ ]:
y_pred2021 = gb_model.predict(test2021)
y_pred2021

array([  3.47558431,   3.47558431,  19.0825347 ,  19.0825347 ,
        57.35973109,  57.35973109,   1.98911177,   3.96035177,
         1.98911177,   3.96035177,  25.99001868,  25.99001868,
        25.60071783,  25.60071783,  27.82730917,  27.82730917,
        40.91601731,  40.91601731,  38.87850213,  38.87850213,
        20.63195102,  20.63195102,   2.13171989,   5.02601219,
         3.91131087,   5.02601219,   3.91131087,   5.0926877 ,
         5.0926877 ,   5.03708761,   3.64620676,   5.81631019,
        11.00113169,   5.81631019,  11.00113169,  10.45015168,
        12.20169207,   7.48403069,   7.48403069,   1.6588559 ,
         1.6588559 ,  12.68724013,  12.68724013,   5.39187485,
        13.02999139,  13.02999139,   4.8332597 ,  14.11894718,
        14.11894718,  26.06439371,  26.06439371,   8.12361581,
         8.12361581,   9.57621891,   9.57621891,   6.60103834,
         5.32043289,   5.32043289,   8.78871719,   8.78871719,
        10.98547137,  10.98547137,   4.9815544 ,   5.24

In [ ]:
fee_pred2021 = pd.DataFrame(y_pred2021)
fee_pred2021 = fee_pred2021.rename(columns={0: 'fee_pred'})

In [ ]:
fee_pred2021

,fee_pred
0,3.475584
1,3.475584
2,19.082535
3,19.082535
4,57.359731
...,...
380,8.461392
381,5.094555
382,4.889024
383,4.889024


In [ ]:
batch2021['fee_pred'] = fee_pred2021
batch2021.columns
batch2021

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_pred
0,32.0,80,80,15500000.0,38000,79,75,69,80,71,...,64,79,78,77,11,9,10,13,13,3.475584
1,32.0,80,80,15500000.0,38000,79,75,69,80,71,...,64,79,78,77,11,9,10,13,13,3.475584
2,31.0,83,83,32000000.0,71000,86,48,74,80,58,...,66,77,83,81,11,14,8,11,10,19.082535
3,31.0,83,83,32000000.0,71000,86,48,74,80,58,...,66,77,83,81,11,14,8,11,10,19.082535
4,21.0,81,89,56500000.0,95000,77,77,70,80,72,...,66,35,24,28,12,9,11,15,12,57.359731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,28.0,77,79,11000000.0,26000,46,30,77,72,32,...,52,80,78,81,11,16,11,15,13,8.461392
381,24.0,59,67,475000.0,6000,34,27,55,39,31,...,40,61,66,71,7,7,5,10,15,5.094555
382,24.0,69,75,2000000.0,7000,28,22,65,62,23,...,31,71,72,70,9,14,10,12,13,4.889024
383,24.0,69,75,2000000.0,7000,28,22,65,62,23,...,31,71,72,70,9,14,10,12,13,4.889024


In [ ]:
result_2021['fee_pred'] = fee_pred2021
result_2021.columns

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season', 'country', 'short_name', 'Name',
       'Team', 'Position', 'Nationality', 'Overall', 'Potential', 'Value',
       'Wage', 'preferred_foot', 'Work Rate', 'Body Type', 'Release Clause',
       'Crossing', 'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'fee_pred'],
      dtype='object')

In [ ]:
new_column_order2021 = ['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period',
       'league_name', 'year', 'season', 'country', 'short_name', 'Name',
       'Team', 'Position', 'Nationality', 'Overall', 'Potential', 'Value',
       'Wage', 'preferred_foot', 'Work Rate', 'Body Type', 'Release Clause',
       'Crossing', 'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'fee_cleaned','fee_pred']
Final2021 = result_2021[new_column_order2021]

In [ ]:
Final2021

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,league_name,year,...,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_cleaned,fee_pred
0,Atlético de Madrid,Daniel Wass,32.0,Central Midfield,Valencia,2.70m,in,Winter,Primera Division,2021,...,79,78,77,11,9,10,13,13,2.70,3.475584
1,Valencia CF,Daniel Wass,32.0,Central Midfield,Atlético Madrid,2.70m,out,Winter,Primera Division,2021,...,79,78,77,11,9,10,13,13,2.70,3.475584
2,Atlético de Madrid,Kieran Trippier,31.0,Right-Back,Newcastle,30.00m,out,Winter,Primera Division,2021,...,77,83,81,11,14,8,11,10,30.00,19.082535
3,Newcastle United,Kieran Trippier,31.0,Right-Back,Atlético Madrid,30.00m,in,Winter,Premier League,2021,...,77,83,81,11,14,8,11,10,30.00,19.082535
4,FC Barcelona,Ferran Torres,21.0,Left Winger,Man City,55.00m,in,Winter,Primera Division,2021,...,35,24,28,12,9,11,15,12,55.00,57.359731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,ACF Fiorentina,Matija Nastasi?,28.0,Centre-Back,FC Schalke 04,500Th.,in,Summer,Serie A,2021,...,80,78,81,11,16,11,15,13,0.50,8.461392
381,ACF Fiorentina,Julián Illanes,24.0,Centre-Back,Pescara,350Th.,out,Summer,Serie A,2021,...,61,66,71,7,7,5,10,15,0.35,5.094555
382,Spezia Calcio,Ardian Ismajli,24.0,Centre-Back,FC Empoli,2.00m,out,Summer,Serie A,2021,...,71,72,70,9,14,10,12,13,2.00,4.889024
383,FC Empoli,Ardian Ismajli,24.0,Centre-Back,Spezia Calcio,2.00m,in,Summer,Serie A,2021,...,71,72,70,9,14,10,12,13,2.00,4.889024


ทำนายราคานักเตะในปี2022

In [ ]:
batch2022 = result_2022.drop(['club_name',
                              'club_involved_name',
                              'Nationality',
                              'transfer_movement',
                              'Id',
                              'Position',
                              'fee_cleaned',
                              'Release Clause',

                              'short_name',
                              'player_name',
                               'Name',
                              'position',
                              'fee',
                              'transfer_period',
                              'league_name',
                              'season',
                              'country',



                              'preferred_foot',
                              'Body Type',



                              'Composure',
                              'Work Rate',




                              'Team',
                              'year',


                              ], axis=1)
batch2022.describe()

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
count,497.000000,497.000000,497.000000,4.970000e+02,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,...,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000
mean,25.301811,74.718310,80.187123,1.555605e+07,40147.283702,57.372233,54.931590,60.778672,69.076459,51.000000,...,62.591549,54.436620,53.591549,55.203219,51.539235,14.776660,14.724346,14.476861,14.931590,15.004024
std,3.641364,6.207202,4.699776,2.074073e+07,46155.285731,17.421954,21.066603,17.182937,11.974359,19.262616,...,12.988680,15.354957,21.502536,23.394947,23.641539,17.342622,16.682836,16.387691,17.083622,17.835380
min,18.000000,54.000000,63.000000,2.300000e+05,500.000000,9.000000,5.000000,8.000000,17.000000,7.000000,...,24.000000,13.000000,9.000000,10.000000,8.000000,5.000000,5.000000,5.000000,5.000000,5.000000
25%,22.000000,71.000000,77.000000,2.900000e+06,14000.000000,48.000000,38.000000,53.000000,65.000000,37.000000,...,53.000000,45.000000,35.000000,32.000000,27.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,25.000000,75.000000,80.000000,8.500000e+06,27000.000000,62.000000,59.000000,64.000000,72.000000,53.000000,...,65.000000,55.000000,60.000000,65.000000,62.000000,10.000000,11.000000,10.000000,11.000000,11.000000
75%,28.000000,78.000000,83.000000,1.850000e+07,50000.000000,72.000000,72.000000,72.000000,77.000000,67.000000,...,74.000000,66.000000,72.000000,75.000000,72.000000,13.000000,13.000000,13.000000,14.000000,14.000000
max,35.000000,92.000000,93.000000,1.375000e+08,310000.000000,84.000000,95.000000,93.000000,92.000000,90.000000,...,88.000000,91.000000,90.000000,89.000000,89.000000,87.000000,83.000000,93.000000,88.000000,90.000000


In [ ]:
batch2022.columns

Index(['age', 'Overall', 'Potential', 'Value', 'Wage', 'Crossing', 'Finishing',
       'Heading accuracy', 'Short passing', 'Volleys', 'Dribbling', 'Curve',
       'Fk accuracy', 'Long passing', 'Ball control', 'Acceleration',
       'Sprint speed', 'Agility', 'Reactions', 'Balance', 'Shot power',
       'Jumping', 'Stamina', 'Strength', 'Long shots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Defensive awareness', 'Standing tackle', 'Sliding tackle', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes'],
      dtype='object')

In [ ]:
batch2022

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Vision,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,28.0,85,86,63000000,220000,84,83,65,81,74,...,85,76,23,31,20,8,14,6,12,10
1,28.0,85,86,63000000,220000,84,83,65,81,74,...,85,76,23,31,20,8,14,6,12,10
2,27.0,78,81,17000000,69000,74,52,60,76,41,...,66,62,74,74,76,8,14,14,13,12
3,33.0,84,84,27500000,74000,45,39,83,73,44,...,50,47,86,85,82,9,11,14,9,7
4,33.0,84,84,27500000,74000,45,39,83,73,44,...,50,47,86,85,82,9,11,14,9,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,26.0,71,78,3400000,27000,26,37,74,69,36,...,48,52,70,70,69,13,12,14,12,13
493,22.0,72,81,4900000,20000,40,24,68,64,31,...,48,45,75,76,73,12,10,6,6,13
494,22.0,72,81,4900000,20000,40,24,68,64,31,...,48,45,75,76,73,12,10,6,6,13
495,27.0,75,78,6500000,17000,65,56,73,70,54,...,65,63,76,76,76,6,10,9,7,9


In [ ]:
test2022 = scale.fit_transform(batch2022)

test2022.shape

(497, 38)

In [ ]:
batch_pca2022 = pca.fit_transform(test2022)
batch_pca2022

array([[-5.31168754,  4.63119543,  2.52277854, ...,  0.03786002,
         0.08154431, -0.15203982],
       [-5.31168754,  4.63119543,  2.52277854, ...,  0.03786002,
         0.08154431, -0.15203982],
       [-1.65842324, -1.00078331,  0.7211952 , ..., -0.31567878,
        -0.11854439,  0.05071849],
       ...,
       [ 2.90859126, -3.88032283,  0.22372169, ..., -0.21371574,
        -0.21760995, -0.10878085],
       [-1.19567843, -1.87558544,  0.6530023 , ...,  0.30610495,
         0.10005389, -0.21171415],
       [ 3.48556189, -3.87249996, -0.85183343, ..., -0.54492476,
        -0.36123088,  0.25489534]])

In [ ]:
y_pred2022 = gb_model.predict(test2022)
y_pred2022

array([20.4761267 , 20.4761267 , 13.81570655, 13.08777775, 13.08777775,
        7.25092452,  4.83749255,  4.97722415,  7.25092452,  4.83749255,
        4.97722415,  3.78968279,  5.79631309,  3.26048895, 28.65606082,
       28.65606082, 43.4399173 , 43.4399173 , 25.82123692, 25.82123692,
        7.75419987,  7.75419987, 40.14796869, 40.14796869, 37.03080157,
       37.03080157,  8.39669051,  8.39669051,  7.90962822,  8.39948297,
        8.39948297,  5.28293617,  9.26651283,  2.68358108,  2.68358108,
        1.53487999, 21.10276225, 21.10276225,  6.97489163, 14.1162983 ,
        6.97489163, 14.1162983 ,  7.10620775, 14.45601364, 19.9561795 ,
       21.80194855, 19.9561795 , 33.38866464, 33.38866464,  2.93361407,
        3.22835771, 11.52784026, 11.67187854, 19.15817711, 19.15817711,
        8.80721261,  8.80721261, 16.96490641, 16.96490641,  5.10652386,
        8.13278206,  9.44983399, 15.16758897, 15.16758897, 19.21710372,
       19.21710372, 12.4392646 , 12.4392646 , 37.93462768, 37.93

In [ ]:
fee_pred2022 = pd.DataFrame(y_pred2022)
fee_pred2022 = fee_pred2022.rename(columns={0: 'fee_pred'})

In [ ]:
fee_pred2022

,fee_pred
0,20.476127
1,20.476127
2,13.815707
3,13.087778
4,13.087778
...,...
492,4.670139
493,9.662507
494,9.662507
495,5.228508


In [ ]:
batch2022['fee_pred'] = fee_pred2022
batch2022.columns
batch2022

,age,Overall,Potential,Value,Wage,Crossing,Finishing,Heading accuracy,Short passing,Volleys,...,Penalties,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_pred
0,28.0,85,86,63000000,220000,84,83,65,81,74,...,76,23,31,20,8,14,6,12,10,20.476127
1,28.0,85,86,63000000,220000,84,83,65,81,74,...,76,23,31,20,8,14,6,12,10,20.476127
2,27.0,78,81,17000000,69000,74,52,60,76,41,...,62,74,74,76,8,14,14,13,12,13.815707
3,33.0,84,84,27500000,74000,45,39,83,73,44,...,47,86,85,82,9,11,14,9,7,13.087778
4,33.0,84,84,27500000,74000,45,39,83,73,44,...,47,86,85,82,9,11,14,9,7,13.087778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,26.0,71,78,3400000,27000,26,37,74,69,36,...,52,70,70,69,13,12,14,12,13,4.670139
493,22.0,72,81,4900000,20000,40,24,68,64,31,...,45,75,76,73,12,10,6,6,13,9.662507
494,22.0,72,81,4900000,20000,40,24,68,64,31,...,45,75,76,73,12,10,6,6,13,9.662507
495,27.0,75,78,6500000,17000,65,56,73,70,54,...,63,76,76,76,6,10,9,7,9,5.228508


In [ ]:
result_2022['fee_pred'] = fee_pred2022
result_2022.columns

Index(['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period', 'fee_cleaned',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Nationality', 'Position', 'Overall', 'Potential', 'Value',
       'Wage', 'preferred_foot', 'Work Rate', 'Body Type', 'Release Clause',
       'Crossing', 'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'fee_pred'],
      dtype='object')

In [ ]:
new_column_order2022 = ['club_name', 'player_name', 'age', 'position', 'club_involved_name',
       'fee', 'transfer_movement', 'transfer_period',
       'league_name', 'year', 'season', 'country', 'short_name', 'Id', 'Name',
       'Team', 'Nationality', 'Position', 'Overall', 'Potential', 'Value',
       'Wage', 'preferred_foot', 'Work Rate', 'Body Type', 'Release Clause',
       'Crossing', 'Finishing', 'Heading accuracy', 'Short passing', 'Volleys',
       'Dribbling', 'Curve', 'Fk accuracy', 'Long passing', 'Ball control',
       'Acceleration', 'Sprint speed', 'Agility', 'Reactions', 'Balance',
       'Shot power', 'Jumping', 'Stamina', 'Strength', 'Long shots',
       'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Composure', 'Defensive awareness', 'Standing tackle', 'Sliding tackle',
       'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning',
       'GK Reflexes', 'fee_cleaned','fee_pred']
Final2022 = result_2022[new_column_order2022]

In [ ]:
Final2022

,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,league_name,year,...,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,fee_cleaned,fee_pred
0,FC Barcelona,Memphis Depay,28.0,Centre-Forward,Atlético Madrid,3.00m,out,Winter,Primera Division,2022,...,23,31,20,8,14,6,12,10,3.0,20.476127
1,Atlético de Madrid,Memphis Depay,28.0,Centre-Forward,Barcelona,3.00m,in,Winter,Primera Division,2022,...,23,31,20,8,14,6,12,10,3.0,20.476127
2,FC Barcelona,Héctor Bellerín,27.0,Right-Back,Sporting CP,1.00m,out,Winter,Primera Division,2022,...,74,74,76,8,14,14,13,12,1.0,13.815707
3,Atlético de Madrid,Felipe,33.0,Centre-Back,Nottm Forest,2.30m,out,Winter,Primera Division,2022,...,86,85,82,9,11,14,9,7,2.3,13.087778
4,Nottingham Forest,Felipe,33.0,Centre-Back,Atlético Madrid,2.30m,in,Winter,Premier League,2022,...,86,85,82,9,11,14,9,7,2.3,13.087778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,SS Lazio,Denis Vavro,26.0,Centre-Back,FC Copenhagen,4.50m,out,Summer,Serie A,2022,...,70,70,69,13,12,14,12,13,4.5,4.670139
493,Atalanta BC,Matteo Lovato,22.0,Centre-Back,Salernitana,7.00m,out,Summer,Serie A,2022,...,75,76,73,12,10,6,6,13,7.0,9.662507
494,US Salernitana 1919,Matteo Lovato,22.0,Centre-Back,Atalanta BC,7.00m,in,Summer,Serie A,2022,...,75,76,73,12,10,6,6,13,7.0,9.662507
495,US Salernitana 1919,Dylan Bronn,27.0,Centre-Back,FC Metz,1.00m,in,Summer,Serie A,2022,...,76,76,76,6,10,9,7,9,1.0,5.228508


In [ ]:
#เซฟไฟล์ไฟนอลแต่ละปีเพื่อนำไป visualization ต่อใน power bi
Final2016.to_csv('Final2016.csv',encoding='utf-8-sig')
Final2017.to_csv('Final2017.csv',encoding='utf-8-sig')
Final2018.to_csv('Final2018.csv',encoding='utf-8-sig')
Final2019.to_csv('Final2019.csv',encoding='utf-8-sig')
Final2020.to_csv('Final2020.csv',encoding='utf-8-sig')
Final2021.to_csv('Final2021.csv',encoding='utf-8-sig')
Final2022.to_csv('Final2022.csv',encoding='utf-8-sig')

ใช้ model เพื่อทำนายราคาในอนาคต 2023

In [ ]:
all23 = '/content/drive/MyDrive/314Final/code model/Book2.csv'
df23 = pd.read_csv(all23, encoding=encoding)

In [ ]:
df23


,Id,Name,Team,Nationality,Position,age,Overall,Potential,preferred_foot,Work Rate,...,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Value,Wage
0,223813,D. van Bruggen,Almere City,Netherlands,CB,27,65,67,Right,Medium/ Medium,...,60,65,64,8,13,9,10,14,750000,2000
1,267776,C. Atemona,Kortrijk,Germany,CB,21,59,72,Right,Medium/ Medium,...,56,60,57,13,9,13,10,14,525000,2000
2,271032,M. Smets,Sint-Truiden,Belgium,CB,19,65,79,Right,Medium/ Medium,...,70,66,62,7,6,10,14,8,1600000,2000
3,269477,Hu Jing,Chengdu Rongcheng,China PR,CB,24,54,61,Right,Medium/ Medium,...,57,56,54,7,9,8,14,9,200000,2000
4,213846,C. Suárez,Palestino,Chile,CB,36,71,71,Right,Medium/ Medium,...,72,72,70,9,12,9,7,14,325000,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18433,277667,E. Bilsel,Antalyaspor,Azerbaijan,LM,19,55,67,Right,Medium/ Low,...,29,33,31,13,9,9,7,7,300000,2000
18434,275190,Zheng Haoqian,Nantong Zhiyun,China PR,ST,24,52,56,Right,Medium/ Low,...,19,16,17,12,7,6,15,13,120000,2000
18435,275352,L. Munakandafa,Notts County,Zimbabwe,ST,19,51,64,Right,Medium/ Low,...,20,23,15,10,7,7,15,13,140000,2000
18436,272743,Shi Lishan,Changchun Yatai,China PR,LB,20,50,55,Right,Medium/ Low,...,41,51,54,7,5,13,9,11,60000,2000


In [ ]:
df23.columns

Index(['Id', 'Name', 'Team', 'Nationality', 'Position', 'age', 'Overall',
       'Potential', 'preferred_foot', 'Work Rate', 'Body Type',
       'Release clause', 'Crossing', 'Finishing', 'Heading accuracy',
       'Short passing', 'Volleys', 'Dribbling', 'Curve', 'Fk accuracy',
       'Long passing', 'Ball control', 'Acceleration', 'Sprint speed',
       'Agility', 'Reactions', 'Balance', 'Shot power', 'Jumping', 'Stamina',
       'Strength', 'Long shots', 'Aggression', 'Interceptions', 'Positioning',
       'Vision', 'Penalties', 'Composure', 'Defensive awareness',
       'Standing tackle', 'Sliding tackle', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Value', 'Wage'],
      dtype='object')

In [ ]:
batch2023 = df23.drop(['Id', 'Name', 'Team', 'Nationality', 'Position',
         'Release clause', 'preferred_foot',
       'Work Rate', 'Body Type','Composure','Team',



                              ], axis=1)
batch2023.describe()

,age,Overall,Potential,Crossing,Finishing,Heading accuracy,Short passing,Volleys,Dribbling,Curve,...,Defensive awareness,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Value,Wage
count,18438.000000,18438.000000,18438.000000,18438.000000,18438.000000,18438.000000,18438.000000,18438.000000,18438.000000,18438.000000,...,18438.000000,18438.000000,18438.000000,18438.000000,18438.000000,18438.000000,18438.000000,18438.000000,1.843800e+04,18438.000000
mean,25.217920,65.787016,71.043551,49.273837,46.293795,51.796778,59.040623,42.444137,55.932368,47.746122,...,46.671548,48.571483,46.368098,16.287070,16.082167,15.982536,16.129895,16.379054,2.805470e+06,8580.532053
std,4.770985,6.780613,6.226371,17.788008,19.687893,17.236675,14.280268,17.546476,18.724899,17.849446,...,20.378308,20.997655,20.518747,17.518893,16.896236,16.644232,17.045743,17.827508,7.537174e+06,18600.837955
min,16.000000,47.000000,49.000000,7.000000,3.000000,5.000000,10.000000,4.000000,4.000000,7.000000,...,3.000000,8.000000,6.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000e+00,0.000000
25%,21.000000,62.000000,67.000000,39.000000,31.000000,44.000000,55.000000,30.000000,50.000000,36.000000,...,29.000000,29.000000,26.000000,8.000000,8.000000,8.000000,8.000000,8.000000,4.750000e+05,1000.000000
50%,25.000000,66.000000,71.000000,53.000000,50.000000,55.000000,62.000000,44.000000,61.000000,50.000000,...,53.000000,56.000000,54.000000,11.000000,11.000000,11.000000,11.000000,11.000000,1.000000e+06,3000.000000
75%,29.000000,70.000000,75.000000,62.000000,62.000000,64.000000,68.000000,56.000000,68.000000,61.000000,...,63.000000,65.000000,63.000000,14.000000,14.000000,14.000000,14.000000,14.000000,2.000000e+06,7000.000000
max,43.000000,91.000000,94.000000,95.000000,96.000000,93.000000,94.000000,91.000000,96.000000,93.000000,...,91.000000,91.000000,90.000000,90.000000,90.000000,91.000000,90.000000,93.000000,1.850000e+08,350000.000000


In [ ]:
batch_column_order2023 = ['age', 'Overall', 'Potential', 'Value', 'Wage', 'Crossing', 'Finishing',
       'Heading accuracy', 'Short passing', 'Volleys', 'Dribbling', 'Curve',
       'Fk accuracy', 'Long passing', 'Ball control', 'Acceleration',
       'Sprint speed', 'Agility', 'Reactions', 'Balance', 'Shot power',
       'Jumping', 'Stamina', 'Strength', 'Long shots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties',
       'Defensive awareness', 'Standing tackle', 'Sliding tackle', 'GK Diving',
       'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes',
       ]
batch2023 = batch2023[batch_column_order2023]

In [ ]:
test2023 = scale.fit_transform(batch2023)

test2023.shape

(18438, 38)

In [ ]:
batch_pca2023 = pca.fit_transform(test2023)
batch_pca2023

array([[-0.7458201 ,  1.938621  , -0.098301  , ...,  0.08204633,
        -0.42614122,  0.16278221],
       [ 1.5971961 ,  2.07206863, -1.79668597, ..., -0.25566522,
        -0.04799924,  0.0591608 ],
       [-0.50348938,  2.2705692 , -0.72118697, ...,  0.4039365 ,
         0.01988286, -0.44926835],
       ...,
       [ 2.47278713, -2.24430031, -3.22134648, ..., -0.04310467,
        -0.10601461, -0.07289101],
       [ 3.97880104,  1.00301095, -4.98607153, ..., -0.09217727,
         0.18500255,  0.31170289],
       [ 3.39294709, -2.87646757, -4.52010707, ..., -0.17258194,
        -0.22040987,  0.20246706]])

In [ ]:
y_pred2023 = gb_model.predict(test2023)
y_pred2023

array([ 6.66838385, 10.33755108, 21.86594547, ...,  6.72901746,
        5.80123122,  6.14737297])

In [ ]:
fee_pred2023 = pd.DataFrame(y_pred2023)
fee_pred2023 = fee_pred2023.rename(columns={0: 'fee_pred'})

In [ ]:
fee_pred2023

,fee_pred
0,6.668384
1,10.337551
2,21.865945
3,4.392934
4,4.916343
...,...
18433,6.417463
18434,5.340177
18435,6.729017
18436,5.801231


In [ ]:
df23['fee_pred'] = fee_pred2023
df23

,Id,Name,Team,Nationality,Position,age,Overall,Potential,preferred_foot,Work Rate,...,Standing tackle,Sliding tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Value,Wage,fee_pred
0,223813,D. van Bruggen,Almere City,Netherlands,CB,27,65,67,Right,Medium/ Medium,...,65,64,8,13,9,10,14,750000,2000,6.668384
1,267776,C. Atemona,Kortrijk,Germany,CB,21,59,72,Right,Medium/ Medium,...,60,57,13,9,13,10,14,525000,2000,10.337551
2,271032,M. Smets,Sint-Truiden,Belgium,CB,19,65,79,Right,Medium/ Medium,...,66,62,7,6,10,14,8,1600000,2000,21.865945
3,269477,Hu Jing,Chengdu Rongcheng,China PR,CB,24,54,61,Right,Medium/ Medium,...,56,54,7,9,8,14,9,200000,2000,4.392934
4,213846,C. Suárez,Palestino,Chile,CB,36,71,71,Right,Medium/ Medium,...,72,70,9,12,9,7,14,325000,2000,4.916343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18433,277667,E. Bilsel,Antalyaspor,Azerbaijan,LM,19,55,67,Right,Medium/ Low,...,33,31,13,9,9,7,7,300000,2000,6.417463
18434,275190,Zheng Haoqian,Nantong Zhiyun,China PR,ST,24,52,56,Right,Medium/ Low,...,16,17,12,7,6,15,13,120000,2000,5.340177
18435,275352,L. Munakandafa,Notts County,Zimbabwe,ST,19,51,64,Right,Medium/ Low,...,23,15,10,7,7,15,13,140000,2000,6.729017
18436,272743,Shi Lishan,Changchun Yatai,China PR,LB,20,50,55,Right,Medium/ Low,...,51,54,7,5,13,9,11,60000,2000,5.801231


In [ ]:
df23.to_csv('Final2023.csv',encoding='utf-8-sig')